In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from copy import deepcopy
import numpy as np
import pytreenet as ptn
import random
from numpy.random import default_rng, Generator
from fractions import Fraction

# Tree Tensor Network Operator for the Toy Model
In the main manuscript we consider a toy Hamiltonian to show the workings of our algorithm to find tree tensor network operators from a Hamiltonian. The toy Hamiltonian is defined on a tree structure as given in Fig.4.1a) in the main manuscript as
$$
H_{\text{toy}}= \sum_{j=1}^4 h_j = Y_2 X_3 X_4 + X_1 Y_2 Y_6 + X_1 Y_2 Z_5 + Z_5 X_7 X_8.
$$
Here we want to show that our implemented algorithm yields the same state diagrams as the ones shown in the main text.

In [3]:
# Building the tree
def construct_reference_tree() -> ptn.TreeTensorNetworkState:
    """
    Generates the desired tree tensor network used as a reference to construct
     the Hamiltonian.
    """
    ttns = ptn.TreeTensorNetworkState()
    # Physical legs come last
    node1, tensor1 = ptn.random_tensor_node((1, 1, 2), identifier="site1")
    node2, tensor2 = ptn.random_tensor_node((1, 1, 1, 2), identifier="site2")
    node3, tensor3 = ptn.random_tensor_node((1, 2), identifier="site3")
    node4, tensor4 = ptn.random_tensor_node((1, 2), identifier="site4")
    node5, tensor5 = ptn.random_tensor_node((1, 1, 1, 2), identifier="site5")
    node6, tensor6 = ptn.random_tensor_node((1, 2), identifier="site6")
    node7, tensor7 = ptn.random_tensor_node((1, 1, 2), identifier="site7")
    node8, tensor8 = ptn.random_tensor_node((1, 2), identifier="site8")
    
    ttns.add_root(node1, tensor1)
    ttns.add_child_to_parent(node2, tensor2, 0, "site1", 0)
    ttns.add_child_to_parent(node3, tensor3, 0, "site2", 1)
    ttns.add_child_to_parent(node4, tensor4, 0, "site2", 2)
    ttns.add_child_to_parent(node5, tensor5, 0, "site1", 1)
    ttns.add_child_to_parent(node6, tensor6, 0, "site5", 1)
    ttns.add_child_to_parent(node7, tensor7, 0, "site5", 2)
    ttns.add_child_to_parent(node8, tensor8, 0, "site7", 1)
    return ttns

In [4]:
def construct_toy_hamiltonian() -> ptn.Hamiltonian:
    paulis = ptn.pauli_matrices()
    conversion_dict = {"X": paulis[0], "Y": paulis[1], "Z": paulis[2], "I2": np.eye(2)}
    
    ham = [{'site8': 'I2', 'site4': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site4': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site8': 'Z', 'site1': 'X', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'X', 'site4': 'Z', 'site6': 'Y', 'site7': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site8': 'I2'}]
    #coeffs = [1, 1, 1, 1]
    
    return ptn.Hamiltonian([ptn.TensorProduct(i) for i in ham], conversion_dictionary=conversion_dict) #, coeffs = [random.randint(1, 100) for _ in range(18)])



In [5]:
def construct_hamiltonian(ham, coeffs_mapping, coeffs= None, ) -> ptn.Hamiltonian:

    if coeffs is None:
        coeffs = [1 for _ in range(len(ham))]
    
    paulis = ptn.pauli_matrices()
    conversion_dict = {"X": paulis[0], "Y": paulis[1], "Z": paulis[2], "I2": np.eye(2)}
    return ptn.Hamiltonian([ptn.TensorProduct(i) for i in ham], conversion_dictionary=conversion_dict, coeffs=coeffs, coeffs_mapping=coeffs_mapping)


In [6]:
def construct_toy_hamiltonian() -> ptn.Hamiltonian:
    paulis = ptn.pauli_matrices()
    conversion_dict = {"X": paulis[0], "Y": paulis[1], "Z": paulis[2], "I2": np.eye(2)}
    term1 = ptn.TensorProduct({"site2": "Y", "site3": "X", "site4": "X"})
    term2 = ptn.TensorProduct({"site1": "X", "site2": "Y", "site6": "Y"})
    term3 = ptn.TensorProduct({"site1": "X", "site2": "Y", "site5": "Z"})
    term4 = ptn.TensorProduct({"site5": "Z", "site7": "X", "site3": "X"})
    terms = [term1, term2, term3, term4]
    coeffs = [(Fraction(2,3),"2"), (Fraction(1,4),"3"), (Fraction(1,3),"4"), (Fraction(1,2),"5")]
    ham_terms =  [(x, y, z) for (x, y), z in zip(coeffs, terms)]
    coeffs_mapping = {"1":1, "2":2, "3":3, "4":4, "5":5}
    
    return ptn.Hamiltonian(ham_terms, conversion_dictionary=conversion_dict ,coeffs_mapping=coeffs_mapping)

In [7]:

#hamiltonian_toy = construct_hamiltonian(ham, ham_dict, coeffs)
hamiltonian_toy = construct_toy_hamiltonian()
ttns_root1 = construct_reference_tree()
hamiltonian1 = hamiltonian_toy.pad_with_identities(ttns_root1)
#hamiltonian1 = hamiltonian_toy

### The Single Term Diagrams

In [8]:
for i, term in enumerate(hamiltonian1.terms):
    single_term_diag = ptn.SingleTermDiagram.from_single_term(term, ttns_root1)
    print(f"State Diagram for term {i}")
    print(single_term_diag)
    print("-----------------")

State Diagram for term 0
Hyperedges: {'site1': 'I2 (2/3 * 2)', 'site2': 'Y (1 * 1)', 'site3': 'X (1 * 1)', 'site4': 'X (1 * 1)', 'site5': 'I2 (1 * 1)', 'site6': 'I2 (1 * 1)', 'site7': 'I2 (1 * 1)', 'site8': 'I2 (1 * 1)'}
Vertices: [('site1', 'site2'), ('site1', 'site5'), ('site2', 'site3'), ('site2', 'site4'), ('site5', 'site6'), ('site5', 'site7'), ('site7', 'site8')]
-----------------
State Diagram for term 1
Hyperedges: {'site1': 'X (1/4 * 3)', 'site2': 'Y (1 * 1)', 'site3': 'I2 (1 * 1)', 'site4': 'I2 (1 * 1)', 'site5': 'I2 (1 * 1)', 'site6': 'Y (1 * 1)', 'site7': 'I2 (1 * 1)', 'site8': 'I2 (1 * 1)'}
Vertices: [('site1', 'site2'), ('site1', 'site5'), ('site2', 'site3'), ('site2', 'site4'), ('site5', 'site6'), ('site5', 'site7'), ('site7', 'site8')]
-----------------
State Diagram for term 2
Hyperedges: {'site1': 'X (1/3 * 4)', 'site2': 'Y (1 * 1)', 'site3': 'I2 (1 * 1)', 'site4': 'I2 (1 * 1)', 'site5': 'Z (1 * 1)', 'site6': 'I2 (1 * 1)', 'site7': 'I2 (1 * 1)', 'site8': 'I2 (1 * 1)'}

We can see that the single term diagrams are exactly the one we depicted in Fig. 4.1b).

### Complete State Diagram

In [9]:
state_diagram1 = ptn.StateDiagram.from_hamiltonian(hamiltonian1, ttns_root1, ptn.state_diagram.TTNOFinder.SGE)
print(len(state_diagram1.get_all_vertices()))
print(state_diagram1)
#ptn.TTNO.from_hamiltonian(hamiltonian1, ttns_root1)
#print(state_diagram1.coeffs, state_diagram1.coeffs_indices)
print(hamiltonian1.coeffs_mapping)

15
hyperedges:
label = I2; corr_site = site1; coeff = 2/3 * 2; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = X; corr_site = site1; coeff = 1 * 1; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = I2; corr_site = site1; coeff = 1/2 * 5; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = Y; corr_site = site2; coeff = 1 * 1; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = Y; corr_site = site2; coeff = 1 * 1; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = I2; corr_site = site2; coeff = 1 * 1; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = X; corr_site = site3; coeff = 1 * 1; connected to ('site2', 'site3'), 
label = I2; corr_site = site3; coeff = 1 * 1; connected to ('site2', 'site3'), 
label = X; corr_site = site4; coeff = 1 * 1; connected to ('site2', 'site4'), 
label = I2; corr_site = site4; coeff = 1 * 1; connected to ('site2', 'site4'), 
label 

In [10]:
state_diagram2 = ptn.StateDiagram.from_hamiltonian(hamiltonian1, ttns_root1, ptn.TTNOFinder.TREE)
print(len(state_diagram2.get_all_vertices()))
print(state_diagram2)
#ptn.TTNO.from_hamiltonian(hamiltonian1, ttns_root1)


15
hyperedges:
label = I2; corr_site = site1; coeff = 2/3 * 2; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = X; corr_site = site1; coeff = 1/4 * 3; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = I2; corr_site = site1; coeff = 1/2 * 5; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = Y; corr_site = site2; coeff = 1 * 1; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = Y; corr_site = site2; coeff = 1 * 1; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = I2; corr_site = site2; coeff = 1 * 1; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = X; corr_site = site3; coeff = 1 * 1; connected to ('site2', 'site3'), 
label = I2; corr_site = site3; coeff = 1 * 1; connected to ('site2', 'site3'), 
label = X; corr_site = site4; coeff = 1 * 1; connected to ('site2', 'site4'), 
label = I2; corr_site = site4; coeff = 1 * 1; connected to ('site2', 'site4'), 
labe

### Compare tensor values

In [16]:
def _find_permutation_rec(ttno, leg_dict, node_id, perm):
    node, _ = ttno[node_id]
    input_index = leg_dict[node_id]
    output_index = input_index + len(ttno.nodes)
    perm.extend([output_index, input_index ])
    if not node.is_leaf():
        for child_id in node.children:
            _find_permutation_rec(ttno, leg_dict, child_id, perm)

In [20]:
leg_dict = {"site1": 0, "site2": 1, "site3": 2, "site4": 3, "site5": 4,
                "site6": 5, "site7": 6, "site8": 7}

original_tensor = hamiltonian1.to_tensor(ttns_root1).operator#.transpose([3,0,4,1,5,2])

ttno = ptn.TTNO.from_hamiltonian(hamiltonian1, ttns_root1, ptn.state_diagram.TTNOFinder.TREE)


#print(ttno.tensors)

contructed_tensor = ttno.completely_contract_tree(to_copy=True)[0]

permutation = []
_find_permutation_rec(ttno, leg_dict, ttno.root_id, permutation)
correct_tensor = original_tensor.transpose(permutation)
#print(permutation)


print("Equality: ", np.allclose(correct_tensor,contructed_tensor))
#print(original_tensor)
print("---------------------------")
#print(contructed_tensor)

Equality:  False
---------------------------


One can see that this state diagram corresponds to the state diagram given in Fig. 4.1c) in the main text.

In [11]:
def obtain_bond_dimensions(ttno: ptn.TTNO) -> np.ndarray:
    """
    Obtains the bond dimensions of a TTN.

    Args:
        ttno (ptn.TTNO): The TTN for which to determine the bond dimensions.

    Returns:
        np.ndarray: A 1D-array containing all bond-dimensions
    """
    dimensions = []
    for node_id in ttno.nodes:
        node = ttno.nodes[node_id]
        if not node.is_root():
            dimensions.append(node.parent_leg_dim())
    return np.asarray(dimensions)

In [12]:
leg_dict = {"site1": 0, "site2": 1, "site3": 2, "site4": 3, "site5": 4,
                "site6": 5, "site7": 6, "site8": 7,"site9": 8}
ttno_ham = ptn.TTNO.from_hamiltonian(hamiltonian1, ttns_root1)
total_tensor = hamiltonian1.to_tensor(ttns_root1).operator
ttno_svd = ptn.TTNO.from_tensor(ttns_root1,
                                total_tensor,
                                leg_dict,
                                mode=ptn.Decomposition.tSVD)
ham_dim = obtain_bond_dimensions(ttno_ham)
svd_dim = obtain_bond_dimensions(ttno_svd)

print("ham_dim",ham_dim,"svd_dim", svd_dim)
if np.any(ham_dim > svd_dim):
    print(ham_dim, svd_dim)

ham_dim [1 1 1 1 1 1 1] svd_dim [1 1 1 1 1 1 1]


## Find Minimum problematic hamiltonian

In [ ]:
ham = [{'site4': 'Z', 'site2': 'Y', 'site8': 'Y', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'Y', 'site1': 'Z', 'site3': 'X', 'site2': 'Z', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Z', 'site8': 'Y', 'site2': 'I2', 'site6': 'X', 'site5': 'I2', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2'}, {'site3': 'I2', 'site4': 'Z', 'site7': 'X', 'site8': 'Z', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site6': 'I2', 'site5': 'Y', 'site1': 'X', 'site8': 'X', 'site4': 'Z', 'site7': 'X', 'site2': 'I2', 'site3': 'I2'}, {'site6': 'I2', 'site1': 'Y', 'site5': 'Z', 'site4': 'I2', 'site2': 'I2', 'site3': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'I2', 'site4': 'X', 'site2': 'X', 'site1': 'X', 'site6': 'I2', 'site5': 'Z', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'X', 'site2': 'Z', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site7': 'I2', 'site2': 'Z', 'site4': 'X', 'site3': 'Y', 'site8': 'I2', 'site1': 'I2', 'site6': 'I2'}, {'site7': 'X', 'site4': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site7': 'X', 'site2': 'I2', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site2': 'I2', 'site6': 'Y', 'site8': 'Z', 'site1': 'Z', 'site3': 'Z', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site8': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site2': 'I2', 'site3': 'I2', 'site6': 'Z', 'site7': 'Z', 'site4': 'Z', 'site8': 'Z', 'site1': 'I2', 'site5': 'I2'}, {'site8': 'Y', 'site6': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site7': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site8': 'I2', 'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}]
coeffs = [-4.67, -2.9633104276996303, 6.680706729782859, -4.30560120789554, 5.202483584181863, 8.505121153194981, -4.011735853871354, -7.643281057886108, 6.157174317449904, 9.561269374270271, 2.9385071692017757, -8.54746490019713, 7.798682301513789, -4.6568899743444785, 9.241682837178718, -6.285495010325322, -5.907195938916027, -4.12665854873854]
leg_dict = {"site1": 0, "site2": 1, "site3": 2, "site4": 3, "site5": 4, "site6": 5, "site7": 6, "site8": 7,"site9": 8}
        
for i in range(len(ham)):
    c = coeffs[i]
    coeffs[i] = coeffs[i]+10
    hamiltonian_temp = construct_hamiltonian(ham,coeffs)
    ttns_root = construct_reference_tree()
    ham_t = hamiltonian_temp.pad_with_identities(ttns_root)

    ttno_ham = ptn.TTNO.from_hamiltonian(ham_t, ttns_root)
    total_tensor = ham_t.to_tensor(ttns_root).operator
    ttno_svd = ptn.TTNO.from_tensor(ttns_root,
                                    total_tensor,
                                    leg_dict,
                                    mode=ptn.Decomposition.tSVD)
    ham_dim = obtain_bond_dimensions(ttno_ham)
    svd_dim = obtain_bond_dimensions(ttno_svd)


    

    if np.any(ham_dim > svd_dim):
        print(i, ham_dim, svd_dim, "still broken for")
    coeffs[i] = c
        

In [ ]:

ham, coeffs = [{'site1': 'X', 'site8': 'Y', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site3': 'Y', 'site8': 'I2', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site8': 'I2', 'site7': 'Z', 'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site6': 'Z', 'site1': 'I2', 'site7': 'I2', 'site5': 'Z', 'site8': 'Y', 'site2': 'X', 'site3': 'I2', 'site4': 'I2'}, {'site6': 'Z', 'site1': 'I2', 'site8': 'X', 'site3': 'Z', 'site2': 'X', 'site7': 'X', 'site4': 'I2', 'site5': 'I2'}, {'site7': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'Z', 'site5': 'X', 'site6': 'X', 'site2': 'Z', 'site8': 'I2', 'site1': 'I2', 'site4': 'I2', 'site7': 'I2'}, {'site4': 'Z', 'site8': 'X', 'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Z', 'site7': 'Z', 'site6': 'Y', 'site3': 'I2', 'site1': 'I2', 'site8': 'X', 'site2': 'I2', 'site5': 'I2'}, {'site5': 'Z', 'site8': 'I2', 'site2': 'I2', 'site4': 'I2', 'site1': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'Z', 'site3': 'Z', 'site2': 'I2', 'site1': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site1': 'X', 'site8': 'Y', 'site4': 'X', 'site2': 'X', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site8': 'X', 'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2'}], [(Fraction(6, 1), '9'), (Fraction(3, 10), '12'), (Fraction(3, 1), '5'), (Fraction(3, 4), '2'), (Fraction(6, 7), '12'), (Fraction(-8, 1), '12'), (Fraction(-9, 10), '4'), (Fraction(1, 4), '10'), (Fraction(1, 1), '5'), (Fraction(1, 1), '4'), (Fraction(4, 9), '5'), (Fraction(-1, 1), '2'), (Fraction(4, 9), '14'), (Fraction(-4, 5), '3')]
ham_dict = {'2': 2.835697909530385, '3': 4.563938233861004, '4': 9.061058628831027, '5': 8.32944716069347, '6': 6.684243698012364, '7': 5.819053198073945, '8': 8.726491608447832, '9': 6.686432680473677, '10': 7.0935373289106955, '11': 4.444348450120315, '12': 7.854479877498686, '13': 9.703525270953671, '14': 9.802210710023699, '15': 9.712784232553258}
leg_dict = {"site1": 0, "site2": 1, "site3": 2, "site4": 3, "site5": 4, "site6": 5, "site7": 6, "site8": 7,"site9": 8}
        

term_eliminated = 0

while True:
    error = False
    for i in range(len(ham)):
        t = ham.pop(i)
        c = coeffs.pop(i)
        
        hamiltonian_temp = construct_hamiltonian(ham,ham_dict,coeffs)
        ttns_root = construct_reference_tree()
        ham_t = hamiltonian_temp.pad_with_identities(ttns_root)

        ttno_ham = ptn.TTNO.from_hamiltonian(ham_t, ttns_root)
        total_tensor = ham_t.to_tensor(ttns_root).operator
        ttno_svd = ptn.TTNO.from_tensor(ttns_root,
                                        total_tensor,
                                        leg_dict,
                                        mode=ptn.Decomposition.tSVD)
        ham_dim = obtain_bond_dimensions(ttno_ham)
        svd_dim = obtain_bond_dimensions(ttno_svd)


        

        if np.any(ham_dim > svd_dim):
            error = True
            term_eliminated += 1
            break
        else:
            ham.insert(i, t)
            coeffs.insert(i, c)
        
    if not error :
        break

print(term_eliminated, " terms eliminated") 
print(ham," , ",coeffs)

In [ ]:
ham, coeffs = [{'site6': 'I2', 'site1': 'I2', 'site2': 'Y', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site8': 'I2', 'site2': 'Y', 'site1': 'X', 'site6': 'I2', 'site4': 'I2', 'site3': 'I2', 'site7': 'I2'}, {'site5': 'I2', 'site3': 'Z', 'site6': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site5': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'X', 'site8': 'X', 'site4': 'Z', 'site7': 'X', 'site3': 'I2', 'site5': 'Z', 'site2': 'I2', 'site6': 'I2'}, {'site3': 'Z', 'site6': 'Z', 'site5': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'I2', 'site2': 'I2', 'site3': 'Y', 'site1': 'Y', 'site4': 'X', 'site8': 'X', 'site7': 'X', 'site5': 'I2'}, {'site8': 'X', 'site7': 'Z', 'site4': 'Z', 'site2': 'I2', 'site1': 'X', 'site5': 'X', 'site3': 'I2', 'site6': 'I2'}, {'site7': 'I2', 'site8': 'I2', 'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site8': 'X', 'site2': 'Y', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'Y', 'site7': 'Z', 'site2': 'I2', 'site8': 'Y', 'site4': 'Y', 'site1': 'Y', 'site3': 'X', 'site6': 'I2'}, {'site2': 'Y', 'site3': 'I2', 'site8': 'Z', 'site7': 'Y', 'site5': 'X', 'site1': 'I2', 'site4': 'I2', 'site6': 'I2'}, {'site5': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'I2', 'site6': 'Y', 'site3': 'I2', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'Z', 'site4': 'I2', 'site6': 'X', 'site2': 'Z', 'site1': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site8': 'I2', 'site1': 'Y', 'site6': 'I2', 'site5': 'I2', 'site3': 'I2', 'site4': 'I2', 'site7': 'I2'}, {'site1': 'Z', 'site2': 'Z', 'site4': 'Y', 'site8': 'X', 'site7': 'I2', 'site5': 'X', 'site6': 'X', 'site3': 'I2'}, {'site3': 'Z', 'site6': 'Y', 'site7': 'X', 'site8': 'I2', 'site1': 'Z', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2'}, {'site6': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'I2', 'site2': 'I2', 'site7': 'Y', 'site8': 'X', 'site3': 'X', 'site4': 'Z', 'site1': 'I2', 'site5': 'I2'}, {'site2': 'I2', 'site5': 'Z', 'site1': 'Y', 'site7': 'I2', 'site8': 'Y', 'site3': 'Z', 'site4': 'I2', 'site6': 'I2'}, {'site4': 'Y', 'site1': 'Z', 'site6': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Z', 'site8': 'X', 'site3': 'Y', 'site1': 'Y', 'site6': 'Y', 'site5': 'X', 'site4': 'I2', 'site7': 'I2'}], [-3, 1, 3, 3, -1, 5, -3, -4, 1, -3, -3, -5, -3, -4, 5, 2, -3, 1, 2, -1, -4, 5, -3]

leg_dict = {"site1": 0, "site2": 1, "site3": 2, "site4": 3, "site5": 4, "site6": 5, "site7": 6, "site8": 7,"site9": 8}
        

term_eliminated = 0
while True:
    error = False
    for i in range(len(ham)):
        t = ham.pop(i)
        c = coeffs.pop(i)
        hamiltonian_temp = construct_hamiltonian(ham, coeffs)
        ttns_root = construct_reference_tree()
        ham_t = hamiltonian_temp.pad_with_identities(ttns_root)

        ttno_ham = ptn.TTNO.from_hamiltonian(ham_t, ttns_root)
        
        total_tensor = ham_t.to_tensor(ttns_root).operator
        ttno_svd = ptn.TTNO.from_tensor(ttns_root,
                                        total_tensor,
                                        leg_dict,
                                        mode=ptn.Decomposition.tSVD)

        contructed_tensor = ttno_ham.completely_contract_tree(to_copy=True)[0]

        permutation = []
        _find_permutation_rec(ttno_ham, leg_dict, ttno_ham.root_id, permutation)
        #print(permutation)
        correct_tensor = total_tensor.transpose(permutation)

        if not np.allclose(correct_tensor,contructed_tensor):
            error = True
            term_eliminated += 1
            break
        else:
            ham.insert(i, t)
            coeffs.insert(i, c)
        
    if not error :
        break

print(term_eliminated, " terms eliminated") 
print(ham, coeffs)

In [12]:
def construct_tree_root_at_5():
    """
    Generates the desired tree tensor network with root at site 5 used as a
     reference to construct the Hamiltonian.
    """
    ttns = ptn.TreeTensorNetworkState()

    # Physical legs come last
    node1, tensor1 = ptn.random_tensor_node((1, 1, 2), identifier="site1")
    node2, tensor2 = ptn.random_tensor_node((1, 1, 1, 2), identifier="site2")
    node3, tensor3 = ptn.random_tensor_node((1, 2), identifier="site3")
    node4, tensor4 = ptn.random_tensor_node((1, 2), identifier="site4")
    node5, tensor5 = ptn.random_tensor_node((1, 1, 1, 2), identifier="site5")
    node6, tensor6 = ptn.random_tensor_node((1, 2), identifier="site6")
    node7, tensor7 = ptn.random_tensor_node((1, 1, 2), identifier="site7")
    node8, tensor8 = ptn.random_tensor_node((1, 2), identifier="site8")

    ttns.add_root(node5, tensor5)
    ttns.add_child_to_parent(node1, tensor1, 0, "site5", 0)
    ttns.add_child_to_parent(node2, tensor2, 0, "site1", 1)
    ttns.add_child_to_parent(node3, tensor3, 0, "site2", 1)
    ttns.add_child_to_parent(node4, tensor4, 0, "site2", 2)
    ttns.add_child_to_parent(node6, tensor6, 0, "site5", 1)
    ttns.add_child_to_parent(node7, tensor7, 0, "site5", 2)
    ttns.add_child_to_parent(node8, tensor8, 0, "site7", 1)
    return ttns

In [ ]:
ttns_root5 = construct_tree_root_at_5()
#hamiltonian5 = hamiltonian_toy.pad_with_identities(ttns_root5)
hamiltonian5 = hamiltonian_toy
state_diagram5_normal = ptn.StateDiagram.from_hamiltonian(hamiltonian5, ttns_root5)
state_diagram5_bipartite = ptn.StateDiagram.from_hamiltonian_bipartite(hamiltonian5, ttns_root5)

#print(state_diagram5_normal)
#print(len(state_diagram5_normal.get_all_vertices()))
print(state_diagram5_bipartite)
print(len(state_diagram5_bipartite.get_all_vertices()))
ptn.TTNO.from_hamiltonian(hamiltonian5, ttns_root5)

### Root at 6
Here we consider the same tree structure but with the root of the tree at node 6 instead of node 1. Note that node 6 was a leaf in the other two trees.

In [47]:
def construct_tree_root_at_6():
    """
    Generates the desired tree tensor network with root at site 6 used as a
     reference to construct the Hamiltonian.
    """
    ttns = ptn.TreeTensorNetworkState()

    # Physical legs come last
    node1, tensor1 = ptn.random_tensor_node((1, 1, 2), identifier="site1")
    node2, tensor2 = ptn.random_tensor_node((1, 1, 1, 2), identifier="site2")
    node3, tensor3 = ptn.random_tensor_node((1, 2), identifier="site3")
    node4, tensor4 = ptn.random_tensor_node((1, 2), identifier="site4")
    node5, tensor5 = ptn.random_tensor_node((1, 1, 1, 2), identifier="site5")
    node6, tensor6 = ptn.random_tensor_node((1, 2), identifier="site6")
    node7, tensor7 = ptn.random_tensor_node((1, 1, 2), identifier="site7")
    node8, tensor8 = ptn.random_tensor_node((1, 2), identifier="site8")

    ttns.add_root(node6, tensor6)
    ttns.add_child_to_parent(node5, tensor5, 0, "site6", 0)
    ttns.add_child_to_parent(node1, tensor1, 0, "site5", 1)
    ttns.add_child_to_parent(node2, tensor2, 0, "site1", 1)
    ttns.add_child_to_parent(node3, tensor3, 0, "site2", 1)
    ttns.add_child_to_parent(node4, tensor4, 0, "site2", 2)
    ttns.add_child_to_parent(node7, tensor7, 0, "site5", 2)
    ttns.add_child_to_parent(node8, tensor8, 0, "site7", 1)
    return ttns

In [ ]:
ttns_root6 = construct_tree_root_at_6()
#hamiltonian6 = hamiltonian_toy.pad_with_identities(ttns_root6)
hamiltonian6 = hamiltonian_toy
state_diagram6 = ptn.StateDiagram.from_hamiltonian(hamiltonian6, ttns_root6)
state_diagram6_bi = ptn.StateDiagram.from_hamiltonian_bipartite(hamiltonian6, ttns_root6)

print(state_diagram6)
print(len(state_diagram6.get_all_vertices()))
print(state_diagram6_bi)
print(len(state_diagram6_bi.get_all_vertices()))

# Tree Tensor Network Operator for a Spin-Boson model

## Building the Hamiltonian
We consider a Heisenberg chain of length $N$ where each site couples to $M$ non-interacting Bosons, i.e.
$$
H = H_S + H_E + H_{SE}.
$$

$$
H_S = -J\sum_{s=0}^{N-2} \langle \vec{\sigma} , \vec{\sigma} \rangle,
$$
where $J$ is the interaction strength and $\vec{\sigma} = \begin{pmatrix} X & Y & Z \end{pmatrix}^T$ is the vector of Pauli operators.
$$
H_E = \sum_{s=0}^{N-1} \sum_{b=0}^{M-1}\omega_{s,b} B_{s,b}^\dagger B_{s,b},
$$
where $\omega_{s,b}$ is boson's characteristic frequency, and $B$ and $B^\dagger$ are the bosonic annihilation and creation operators.
$$
H_{SE} = \sum_{s=0}^{N-1} \sum_{b=0}^{M-1}\left( g_{s,b} Z_s B_{s,b} + \text{h.c.} \right),
$$
where $g_{s,b}$ is the coupling strength of spin $s$ with boson $(s,b)$.

## Matrix Product Form

## Fork Tree Product Form

# Star Shape Tree

## The Tree Tensor Network Operator

## Small Example to Check Equality
The above size of the TTN will cause the generation of a very large tensor, if the operators are contracted.
To check that the TTNO actually correspond to the Hamiltonians, we will use an example with fewer sites.

### Test Problematic Hamiltonians

In [31]:
input_string = """  0%|                                                                                              | 0/30 [00:00<?, ?it/s]num:  0
num:  500
  3%|██▊                                                                                   | 1/30 [00:18<09:08, 18.91s/it]num:  0
num:  500
  7%|█████▋                                                                                | 2/30 [00:41<09:45, 20.92s/it]num:  0
num:  500
 10%|████████▌                                                                             | 3/30 [01:06<10:18, 22.90s/it]num:  0
num:  500
 13%|███████████▍                                                                          | 4/30 [01:34<10:50, 25.03s/it]num:  0
num:  0
num:  500
 17%|██████████████▎                                                                       | 5/30 [02:06<11:23, 27.33s/it]num:  0
num:  500
 20%|█████████████████▏                                                                    | 6/30 [02:40<11:49, 29.55s/it]num:  0
num:  500
 23%|████████████████████                                                                  | 7/30 [03:18<12:22, 32.29s/it]num:  0
num:  500
 27%|██████████████████████▉                                                               | 8/30 [03:57<12:41, 34.61s/it]num:  0
num:  500
num:  500
num:  500
 30%|█████████████████████████▊                                                            | 9/30 [04:40<12:58, 37.07s/it]num:  0
num:  500
 33%|████████████████████████████▎                                                        | 10/30 [05:25<13:12, 39.63s/it]num:  0
num:  500
 37%|███████████████████████████████▏                                                     | 11/30 [06:13<13:23, 42.31s/it]num:  0
num:  500
 40%|██████████████████████████████████                                                   | 12/30 [07:05<13:32, 45.14s/it]num:  0
num:  500
 43%|████████████████████████████████████▊                                                | 13/30 [07:59<13:34, 47.93s/it]num:  0
num:  500
Error happened!! Total difference:  1
[{'site7': 'X', 'site6': 'X', 'site1': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site2': 'X', 'site1': 'Y', 'site6': 'Y', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site4': 'I2', 'site1': 'X', 'site2': 'I2', 'site7': 'X', 'site6': 'Y', 'site3': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site3': 'X', 'site4': 'I2', 'site2': 'Y', 'site1': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site7': 'Y', 'site3': 'X', 'site8': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site7': 'I2', 'site6': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site2': 'I2', 'site1': 'Z', 'site8': 'Y', 'site5': 'Y', 'site4': 'Y', 'site7': 'I2', 'site6': 'I2'}, {'site6': 'X', 'site1': 'I2', 'site8': 'I2', 'site2': 'Y', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site6': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site1': 'Z', 'site8': 'Y', 'site3': 'I2', 'site7': 'Z', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site3': 'Y', 'site1': 'Y', 'site2': 'I2', 'site5': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Y', 'site4': 'Z', 'site6': 'X', 'site5': 'Z', 'site2': 'Z', 'site3': 'Y', 'site8': 'I2', 'site7': 'I2'}, {'site8': 'Y', 'site5': 'X', 'site3': 'I2', 'site6': 'I2', 'site7': 'Z', 'site2': 'Z', 'site1': 'I2', 'site4': 'I2'}]
[-4.03, 4.62, 2.61, -2.64, -9.67, 4.9, 8.16, 6.65, 6.06, 7.9, 7.9, 7.89, -6.53, -3.84] 
 [-4.03, 4.62, 2.61, -2.64, -9.67, 4.9, 8.16, 6.65, 6.06, 7.9, 7.9, 7.89, -6.53, -3.84]
 47%|███████████████████████████████████████▋                                             | 14/30 [08:56<13:31, 50.70s/it]num:  0
Error happened!! Total difference:  2
[{'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'I2', 'site4': 'Z', 'site6': 'I2', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Z', 'site5': 'Z', 'site4': 'X', 'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site5': 'Y', 'site8': 'I2', 'site3': 'Z', 'site7': 'X', 'site6': 'Z', 'site4': 'Z', 'site1': 'I2', 'site2': 'I2'}, {'site5': 'Y', 'site4': 'X', 'site1': 'Y', 'site3': 'I2', 'site2': 'Y', 'site6': 'Y', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'X', 'site2': 'X', 'site3': 'Z', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site7': 'Y', 'site6': 'Z', 'site1': 'Y', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site7': 'I2', 'site3': 'X', 'site4': 'Z', 'site8': 'I2', 'site1': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site2': 'Z', 'site1': 'X', 'site3': 'Z', 'site6': 'Z', 'site7': 'X', 'site5': 'I2', 'site8': 'X', 'site4': 'I2'}, {'site3': 'I2', 'site2': 'Y', 'site6': 'X', 'site7': 'I2', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site7': 'I2', 'site5': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'X', 'site6': 'I2', 'site3': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site8': 'I2', 'site7': 'Z', 'site2': 'Y', 'site5': 'I2', 'site1': 'X', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2'}, {'site5': 'Y', 'site8': 'Z', 'site1': 'Y', 'site2': 'X', 'site6': 'Z', 'site7': 'I2', 'site3': 'I2', 'site4': 'I2'}]
[-3.4, 7.75, 2.34, 6.71, 8.27, -7.72, -1.78, 7.99, -9.9, 5.87, 8.28, -3.4, -7.47, 9.24, -2.33] 
 [-3.4, 7.75, 2.34, 6.71, 8.27, -7.72, -1.78, 7.99, -9.9, 5.87, 8.28, -3.4, -7.47, 9.24, -2.33]
Error happened!! Total difference:  3
[{'site4': 'Y', 'site5': 'I2', 'site7': 'Z', 'site2': 'Y', 'site1': 'I2', 'site3': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site6': 'X', 'site2': 'X', 'site7': 'I2', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site3': 'I2', 'site6': 'I2', 'site1': 'I2', 'site5': 'X', 'site4': 'X', 'site2': 'Y', 'site7': 'I2'}, {'site1': 'I2', 'site4': 'X', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'Y', 'site1': 'Y', 'site5': 'I2', 'site4': 'Z', 'site8': 'Z', 'site2': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site2': 'Z', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'I2', 'site4': 'X', 'site5': 'X', 'site6': 'X', 'site8': 'I2', 'site2': 'Y', 'site1': 'I2', 'site7': 'I2'}, {'site2': 'X', 'site4': 'X', 'site1': 'X', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site7': 'Z', 'site8': 'Z', 'site5': 'X', 'site1': 'Y', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2'}, {'site4': 'X', 'site6': 'Y', 'site8': 'X', 'site5': 'I2', 'site3': 'I2', 'site1': 'Y', 'site7': 'X', 'site2': 'I2'}, {'site8': 'I2', 'site7': 'I2', 'site4': 'X', 'site1': 'Y', 'site2': 'Z', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Z', 'site8': 'Y', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site8': 'X', 'site1': 'Y', 'site6': 'Z', 'site3': 'X', 'site7': 'Z', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2'}]
[6.21, 5.84, -3.58, -8.07, 1.74, -1.8, 4.5, -7.97, -5.38, -2.7, -8.07, 7.06, -7.51, 5.49, 7.74] 
 [6.21, 5.84, -3.58, -8.07, 1.74, -1.8, 4.5, -7.97, -5.38, -2.7, -8.07, 7.06, -7.51, 5.49, 7.74]
num:  500
 50%|██████████████████████████████████████████▌                                          | 15/30 [09:56<13:23, 53.53s/it]num:  0
num:  500
num:  500
Error happened!! Total difference:  4
[{'site5': 'I2', 'site4': 'Z', 'site1': 'I2', 'site6': 'X', 'site2': 'I2', 'site3': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Y', 'site5': 'Z', 'site3': 'I2', 'site7': 'Y', 'site2': 'I2', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site5': 'Y', 'site6': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site1': 'I2', 'site8': 'I2', 'site3': 'X', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site3': 'I2', 'site8': 'X', 'site1': 'Z', 'site6': 'Y', 'site4': 'Z', 'site5': 'Y', 'site2': 'I2', 'site7': 'I2'}, {'site8': 'Y', 'site5': 'I2', 'site6': 'Z', 'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site7': 'I2'}, {'site7': 'Z', 'site3': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'Y', 'site7': 'I2', 'site5': 'Y', 'site2': 'Z', 'site6': 'Y', 'site1': 'I2', 'site4': 'I2', 'site8': 'I2'}, {'site1': 'X', 'site5': 'I2', 'site7': 'X', 'site6': 'X', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site8': 'I2'}, {'site4': 'X', 'site5': 'I2', 'site3': 'Y', 'site6': 'I2', 'site1': 'I2', 'site2': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Z', 'site3': 'I2', 'site5': 'Z', 'site1': 'Y', 'site4': 'Y', 'site2': 'Z', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site6': 'X', 'site1': 'X', 'site5': 'Y', 'site4': 'X', 'site8': 'Y', 'site3': 'Y', 'site2': 'I2', 'site7': 'I2'}, {'site1': 'X', 'site7': 'Z', 'site5': 'X', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'Z', 'site8': 'Z', 'site6': 'I2', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}]
[-7.03, -6.56, 4.02, -8.45, 6.46, -3.2, 7.15, 3.42, -4.54, 9.83, -8.01, 1.77, -3.59, -8.01, 5.97, 1.68] 
 [-7.03, -6.56, 4.02, -8.45, 6.46, -3.2, 7.15, 3.42, -4.54, 9.83, -8.01, 1.77, -3.59, -8.01, 5.97, 1.68]
 53%|█████████████████████████████████████████████▎                                       | 16/30 [11:00<13:10, 56.45s/it]num:  0
[{'site6': 'Z', 'site3': 'X', 'site2': 'X', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site2': 'Y', 'site7': 'X', 'site3': 'I2', 'site1': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site4': 'Z', 'site7': 'Y', 'site8': 'Y', 'site6': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2'}, {'site3': 'Y', 'site1': 'I2', 'site4': 'Y', 'site8': 'Y', 'site2': 'Y', 'site7': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site4': 'Z', 'site8': 'Z', 'site1': 'Y', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site2': 'X', 'site5': 'I2', 'site3': 'I2', 'site8': 'Y', 'site1': 'X', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site8': 'I2', 'site4': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site6': 'I2', 'site3': 'Y', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'Y', 'site8': 'Z', 'site4': 'Z', 'site2': 'Z', 'site6': 'Y', 'site7': 'X', 'site1': 'I2', 'site3': 'I2'}, {'site7': 'I2', 'site4': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site3': 'X', 'site8': 'Z', 'site2': 'X', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site4': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site8': 'Y', 'site3': 'Z', 'site6': 'X', 'site2': 'I2', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2'}, {'site4': 'I2', 'site3': 'I2', 'site2': 'Z', 'site5': 'Z', 'site8': 'X', 'site1': 'Y', 'site6': 'I2', 'site7': 'I2'}, {'site7': 'Z', 'site4': 'Z', 'site3': 'Y', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}]
[4.75, 5.76, 4.79, -7.85, -6.43, -3.66, 7.75, 7.22, -6.29, 3.76, -5.98, -3.16, 2.52, -3.18, -2.75, 5.28, -5.07]
Difference is:  [11  4  4 10  4  8  4]  ----  [10  4  4 10  4  8  4]
Bond Total difference:  1
coefficients:  [4.75, 5.76, 4.79, -7.85, -6.43, -3.66, 7.75, 7.22, -6.29, 3.76, -5.98, -3.16, 2.52, -3.18, -2.75, 5.28, -5.07]
num:  500
num:  500
 57%|████████████████████████████████████████████████▏                                    | 17/30 [12:03<12:42, 58.65s/it]num:  0
num:  0
num:  0
num:  500
 60%|███████████████████████████████████████████████████                                  | 18/30 [13:10<12:13, 61.10s/it]num:  0
num:  0
num:  500
 63%|█████████████████████████████████████████████████████▊                               | 19/30 [14:25<11:57, 65.20s/it]num:  0
num:  500
num:  500
num:  500
Error happened!! Total difference:  5
[{'site8': 'X', 'site4': 'X', 'site3': 'X', 'site5': 'Z', 'site6': 'Z', 'site1': 'Z', 'site2': 'Y', 'site7': 'I2'}, {'site7': 'X', 'site1': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'Y', 'site2': 'Z', 'site1': 'Y', 'site7': 'Y', 'site6': 'Y', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2'}, {'site5': 'I2', 'site8': 'I2', 'site7': 'Y', 'site6': 'Z', 'site3': 'X', 'site1': 'X', 'site2': 'X', 'site4': 'I2'}, {'site8': 'Y', 'site5': 'X', 'site4': 'Y', 'site1': 'Z', 'site6': 'I2', 'site2': 'I2', 'site3': 'I2', 'site7': 'I2'}, {'site7': 'X', 'site1': 'Y', 'site5': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site1': 'X', 'site7': 'X', 'site8': 'Y', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site7': 'Z', 'site4': 'Z', 'site5': 'Y', 'site3': 'X', 'site6': 'X', 'site8': 'X', 'site2': 'Z', 'site1': 'I2'}, {'site8': 'Z', 'site5': 'Z', 'site4': 'X', 'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site8': 'X', 'site3': 'I2', 'site4': 'X', 'site7': 'Z', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site4': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'I2', 'site3': 'X', 'site6': 'Y', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'X', 'site8': 'I2', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site7': 'Z', 'site1': 'Z', 'site2': 'Y', 'site4': 'I2', 'site8': 'X', 'site5': 'Y', 'site3': 'Y', 'site6': 'I2'}, {'site4': 'Z', 'site3': 'Y', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site8': 'X', 'site7': 'I2', 'site5': 'Y', 'site6': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2'}, {'site2': 'X', 'site8': 'Y', 'site1': 'Y', 'site4': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site4': 'I2', 'site5': 'I2', 'site8': 'Y', 'site2': 'Z', 'site1': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2'}]
[4.76, -6.74, 6.46, 2.37, -3.7, -4.66, 8.85, 8.07, -3.1, 4.95, -6.82, 8.19, -4.71, -2.04, 6.46, -7.84, 2.56, -3.95, 8.24, -5.77] 
 [4.76, -6.74, 6.46, 2.37, -3.7, -4.66, 8.85, 8.07, -3.1, 4.95, -6.82, 8.19, -4.71, -2.04, 6.46, -7.84, 2.56, -3.95, 8.24, -5.77]
 67%|████████████████████████████████████████████████████████▋                            | 20/30 [15:42<11:26, 68.69s/it]num:  0
num:  0
Error happened!! Total difference:  6
[{'site7': 'I2', 'site3': 'Y', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site7': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site7': 'X', 'site5': 'X', 'site8': 'I2', 'site2': 'X', 'site1': 'I2', 'site4': 'I2', 'site6': 'I2'}, {'site4': 'I2', 'site3': 'Y', 'site6': 'I2', 'site5': 'X', 'site2': 'Z', 'site8': 'X', 'site7': 'Y', 'site1': 'I2'}, {'site6': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site4': 'Z', 'site1': 'X', 'site5': 'X', 'site6': 'Z', 'site2': 'X', 'site3': 'I2', 'site7': 'I2'}, {'site7': 'Y', 'site4': 'X', 'site2': 'X', 'site1': 'Z', 'site5': 'I2', 'site3': 'Y', 'site6': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site3': 'I2', 'site6': 'Z', 'site1': 'I2', 'site2': 'X', 'site8': 'X', 'site4': 'I2', 'site5': 'I2'}, {'site3': 'X', 'site6': 'I2', 'site7': 'Z', 'site1': 'Z', 'site2': 'Y', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site1': 'Z', 'site2': 'Y', 'site3': 'Z', 'site7': 'Z', 'site4': 'Y', 'site6': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site8': 'Y', 'site4': 'Z', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site2': 'Z', 'site8': 'Z', 'site4': 'X', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site7': 'I2', 'site8': 'X', 'site4': 'X', 'site2': 'Z', 'site6': 'I2', 'site1': 'X', 'site3': 'I2', 'site5': 'I2'}, {'site7': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site7': 'I2', 'site5': 'X', 'site6': 'X', 'site8': 'I2', 'site1': 'Y', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2'}, {'site6': 'X', 'site4': 'X', 'site1': 'Y', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site2': 'Z', 'site1': 'Z', 'site8': 'Z', 'site5': 'X', 'site3': 'X', 'site7': 'Z', 'site6': 'I2'}, {'site3': 'Y', 'site7': 'Y', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site1': 'X', 'site5': 'Z', 'site7': 'Y', 'site3': 'I2', 'site6': 'Y', 'site2': 'I2', 'site4': 'I2', 'site8': 'I2'}, {'site4': 'I2', 'site8': 'Z', 'site7': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site7': 'Y', 'site6': 'I2', 'site8': 'I2', 'site5': 'I2', 'site4': 'X', 'site2': 'Z', 'site1': 'I2', 'site3': 'I2'}]
[8.72, -1.61, -3.92, 7.77, 2.19, 8.06, 7.13, 4.23, 8.62, -4.19, -2.15, 6.83, 5.34, 9.23, 4.43, 2.19, 4.91, 8.32, -6.41, -4.79, -2.78] 
 [8.72, -1.61, -3.92, 7.77, 2.19, 8.06, 7.13, 4.23, 8.62, -4.19, -2.15, 6.83, 5.34, 9.23, 4.43, 2.19, 4.91, 8.32, -6.41, -4.79, -2.78]
num:  500
num:  500
 70%|███████████████████████████████████████████████████████████▍                         | 21/30 [17:02<10:49, 72.14s/it]num:  0
num:  500
num:  500
Error happened!! Total difference:  7
[{'site3': 'X', 'site6': 'X', 'site2': 'Z', 'site5': 'X', 'site4': 'Z', 'site7': 'I2', 'site8': 'Z', 'site1': 'I2'}, {'site8': 'I2', 'site5': 'X', 'site3': 'Z', 'site7': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site6': 'I2'}, {'site1': 'Z', 'site6': 'X', 'site8': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site5': 'X', 'site3': 'Y', 'site8': 'Z', 'site7': 'Y', 'site1': 'Z', 'site4': 'I2', 'site2': 'I2', 'site6': 'I2'}, {'site8': 'Z', 'site3': 'Z', 'site2': 'X', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'I2', 'site3': 'Z', 'site6': 'I2', 'site2': 'I2', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site2': 'Z', 'site3': 'X', 'site7': 'Y', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site1': 'Y', 'site6': 'Y', 'site2': 'I2', 'site3': 'X', 'site7': 'I2', 'site5': 'Y', 'site8': 'Z', 'site4': 'I2'}, {'site1': 'Y', 'site3': 'Z', 'site5': 'Y', 'site4': 'Z', 'site8': 'X', 'site7': 'I2', 'site2': 'I2', 'site6': 'I2'}, {'site8': 'Y', 'site7': 'Y', 'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2'}, {'site8': 'Y', 'site5': 'I2', 'site7': 'Z', 'site4': 'X', 'site3': 'Z', 'site1': 'I2', 'site2': 'I2', 'site6': 'I2'}, {'site5': 'X', 'site3': 'I2', 'site8': 'X', 'site4': 'X', 'site6': 'Z', 'site2': 'X', 'site1': 'X', 'site7': 'I2'}, {'site1': 'X', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site4': 'X', 'site5': 'X', 'site7': 'Y', 'site8': 'Z', 'site3': 'I2', 'site1': 'I2', 'site6': 'I2'}, {'site4': 'Z', 'site7': 'I2', 'site6': 'I2', 'site1': 'X', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site6': 'X', 'site8': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site5': 'Z', 'site7': 'Z', 'site1': 'Z', 'site2': 'X', 'site3': 'Y', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'I2', 'site5': 'Z', 'site7': 'Z', 'site6': 'Y', 'site8': 'X', 'site4': 'Z', 'site1': 'I2', 'site2': 'I2'}, {'site5': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'X', 'site1': 'I2', 'site8': 'I2', 'site6': 'Z', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site8': 'Z', 'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2'}]
[-5.53, 3.07, 8.72, -5.69, 4.39, 3.98, -9.15, 2.26, -6.09, 4.53, 5.92, 2.53, 6.19, 4.16, -6.12, -4.31, -3.41, 9.54, -4.31, 3.67, 4.2, -8.56] 
 [-5.53, 3.07, 8.72, -5.69, 4.39, 3.98, -9.15, 2.26, -6.09, 4.53, 5.92, 2.53, 6.19, 4.16, -6.12, -4.31, -3.41, 9.54, -4.31, 3.67, 4.2, -8.56]
 73%|██████████████████████████████████████████████████████████████▎                      | 22/30 [18:25<10:03, 75.42s/it]num:  0
Error happened!! Total difference:  8
[{'site6': 'Z', 'site5': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site6': 'X', 'site1': 'Z', 'site5': 'X', 'site2': 'Z', 'site3': 'X', 'site4': 'I2', 'site7': 'I2'}, {'site7': 'Z', 'site2': 'X', 'site5': 'Z', 'site3': 'Y', 'site1': 'I2', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site1': 'Y', 'site3': 'I2', 'site5': 'Y', 'site7': 'Y', 'site4': 'Z', 'site8': 'Z', 'site2': 'I2', 'site6': 'I2'}, {'site6': 'X', 'site8': 'Z', 'site5': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site7': 'I2'}, {'site4': 'X', 'site7': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site2': 'X', 'site4': 'X', 'site7': 'X', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'I2', 'site4': 'X', 'site1': 'X', 'site6': 'X', 'site8': 'X', 'site2': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site1': 'I2', 'site4': 'Y', 'site7': 'I2', 'site6': 'X', 'site3': 'X', 'site8': 'I2', 'site2': 'I2', 'site5': 'I2'}, {'site3': 'X', 'site5': 'I2', 'site4': 'X', 'site2': 'I2', 'site7': 'X', 'site8': 'X', 'site1': 'I2', 'site6': 'I2'}, {'site3': 'X', 'site7': 'I2', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site3': 'I2', 'site4': 'I2', 'site6': 'Z', 'site2': 'I2', 'site5': 'Z', 'site1': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site6': 'Z', 'site1': 'X', 'site4': 'I2', 'site8': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2'}, {'site5': 'Z', 'site6': 'Z', 'site4': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Z', 'site1': 'I2', 'site3': 'I2', 'site6': 'I2', 'site4': 'X', 'site8': 'I2', 'site2': 'I2', 'site5': 'I2'}, {'site8': 'Y', 'site7': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site6': 'Y', 'site2': 'Z', 'site4': 'I2', 'site3': 'X', 'site8': 'X', 'site1': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site6': 'Y', 'site3': 'I2', 'site5': 'X', 'site2': 'I2', 'site7': 'I2', 'site4': 'I2', 'site1': 'Y', 'site8': 'I2'}, {'site3': 'Z', 'site1': 'X', 'site7': 'Y', 'site5': 'X', 'site2': 'X', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site1': 'Y', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'X', 'site5': 'I2', 'site1': 'Y', 'site8': 'Y', 'site6': 'Z', 'site2': 'Y', 'site4': 'Y', 'site3': 'I2'}, {'site7': 'Z', 'site1': 'Y', 'site8': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}]
[9.09, -9.72, -9.92, -7.28, -6.78, -5.85, -8.4, -2.02, 7.55, 5.11, 6.01, -4.65, -9.5, 9.09, 9.96, -3.49, -2.04, 2.15, -7.74, -3.74, 8.95, -4.23, 5.25] 
 [9.09, -9.72, -9.92, -7.28, -6.78, -5.85, -8.4, -2.02, 7.55, 5.11, 6.01, -4.65, -9.5, 9.09, 9.96, -3.49, -2.04, 2.15, -7.74, -3.74, 8.95, -4.23, 5.25]
Error happened!! Total difference:  9
[{'site4': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'Y', 'site1': 'I2', 'site8': 'X', 'site3': 'I2', 'site2': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site4': 'X', 'site8': 'Y', 'site7': 'I2', 'site1': 'I2', 'site2': 'Y', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site5': 'X', 'site6': 'Z', 'site2': 'X', 'site1': 'X', 'site7': 'I2', 'site3': 'Y', 'site4': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site3': 'I2', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Z', 'site4': 'Y', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site7': 'Z', 'site1': 'I2', 'site3': 'X', 'site4': 'I2', 'site8': 'X', 'site2': 'I2', 'site6': 'I2'}, {'site8': 'Y', 'site6': 'Z', 'site7': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2'}, {'site3': 'X', 'site8': 'X', 'site2': 'X', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site6': 'Y', 'site3': 'Y', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site2': 'Y', 'site1': 'I2', 'site7': 'I2', 'site8': 'Y', 'site3': 'Z', 'site5': 'Z', 'site6': 'Z', 'site4': 'I2'}, {'site6': 'X', 'site5': 'Z', 'site2': 'I2', 'site8': 'X', 'site1': 'Z', 'site7': 'X', 'site4': 'X', 'site3': 'I2'}, {'site1': 'X', 'site4': 'I2', 'site6': 'Y', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'I2', 'site6': 'I2', 'site7': 'X', 'site8': 'Z', 'site1': 'I2', 'site3': 'X', 'site2': 'I2', 'site5': 'I2'}, {'site6': 'Z', 'site8': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site5': 'Z', 'site8': 'X', 'site4': 'Y', 'site7': 'Z', 'site1': 'Y', 'site2': 'I2', 'site3': 'I2', 'site6': 'I2'}, {'site3': 'Y', 'site7': 'Y', 'site2': 'X', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site2': 'X', 'site8': 'Z', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site7': 'Y', 'site3': 'Z', 'site5': 'I2', 'site6': 'Y', 'site8': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2'}, {'site1': 'Y', 'site6': 'Y', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}]
[2.93, 5.12, -3.48, -8.5, -7.01, 7.22, 3.39, 4.89, 6.93, 5.51, -8.96, 7.16, 2.93, -7.66, -2.13, 9.86, 2.38, 7.3, -8.96, 4.88, 7.8, 7.45, -3.25] 
 [2.93, 5.12, -3.48, -8.5, -7.01, 7.22, 3.39, 4.89, 6.93, 5.51, -8.96, 7.16, 2.93, -7.66, -2.13, 9.86, 2.38, 7.3, -8.96, 4.88, 7.8, 7.45, -3.25]
num:  500
num:  500
num:  500
Error happened!! Total difference:  10
[{'site7': 'Z', 'site1': 'I2', 'site6': 'I2', 'site4': 'I2', 'site5': 'Y', 'site2': 'Y', 'site3': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site5': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Z', 'site4': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'X', 'site5': 'Z', 'site3': 'Z', 'site4': 'Y', 'site6': 'Z', 'site1': 'Y', 'site2': 'X', 'site7': 'I2'}, {'site2': 'Z', 'site6': 'I2', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'X', 'site8': 'I2', 'site4': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'X', 'site4': 'X', 'site2': 'I2', 'site6': 'Y', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site7': 'Z', 'site1': 'Y', 'site6': 'Y', 'site4': 'I2', 'site5': 'Y', 'site3': 'I2', 'site2': 'Z', 'site8': 'I2'}, {'site7': 'I2', 'site5': 'I2', 'site3': 'Z', 'site8': 'I2', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site6': 'I2'}, {'site7': 'X', 'site1': 'X', 'site3': 'Y', 'site6': 'Z', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site3': 'Z', 'site1': 'I2', 'site2': 'Z', 'site4': 'Z', 'site6': 'Z', 'site5': 'X', 'site8': 'Z', 'site7': 'I2'}, {'site4': 'Z', 'site7': 'X', 'site6': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site7': 'X', 'site8': 'Y', 'site5': 'X', 'site2': 'X', 'site6': 'Y', 'site1': 'X', 'site3': 'I2', 'site4': 'I2'}, {'site6': 'X', 'site2': 'I2', 'site5': 'Y', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'I2', 'site1': 'Y', 'site6': 'X', 'site5': 'I2', 'site2': 'I2', 'site3': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'I2', 'site6': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site5': 'Z', 'site1': 'Y', 'site3': 'I2', 'site4': 'Y', 'site7': 'Y', 'site2': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site1': 'X', 'site3': 'X', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'X', 'site2': 'Z', 'site4': 'Y', 'site7': 'Y', 'site8': 'I2', 'site6': 'X', 'site1': 'I2', 'site3': 'I2'}, {'site8': 'Y', 'site2': 'Y', 'site5': 'Z', 'site4': 'I2', 'site3': 'I2', 'site7': 'Y', 'site1': 'I2', 'site6': 'I2'}]
[-9.18, 6.28, 3.54, 9.69, -7.9, 9.22, 9.89, -3.56, 1.79, 7.04, 8.47, 9.45, 5.71, -5.35, 1.67, 9.04, -5.97, 6.76, 6.24, 6.28, 8.8, -1.78, -5.1] 
 [-9.18, 6.28, 3.54, 9.69, -7.9, 9.22, 9.89, -3.56, 1.79, 7.04, 8.47, 9.45, 5.71, -5.35, 1.67, 9.04, -5.97, 6.76, 6.24, 6.28, 8.8, -1.78, -5.1]
 77%|█████████████████████████████████████████████████████████████████▏                   | 23/30 [19:52<09:11, 78.86s/it]num:  0
num:  0
num:  0
num:  500
num:  500
num:  500
num:  500
num:  500
num:  500
 80%|████████████████████████████████████████████████████████████████████                 | 24/30 [21:23<08:14, 82.43s/it]num:  0
Error happened!! Total difference:  11
[{'site6': 'Z', 'site5': 'I2', 'site1': 'Z', 'site2': 'Z', 'site7': 'Z', 'site3': 'I2', 'site4': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site1': 'I2', 'site4': 'I2', 'site8': 'Y', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site8': 'I2', 'site5': 'I2', 'site3': 'X', 'site4': 'X', 'site6': 'Z', 'site7': 'Z', 'site1': 'Y', 'site2': 'I2'}, {'site4': 'I2', 'site7': 'Y', 'site6': 'I2', 'site3': 'X', 'site5': 'Y', 'site1': 'X', 'site2': 'I2', 'site8': 'I2'}, {'site2': 'Z', 'site1': 'Z', 'site4': 'X', 'site5': 'Z', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'I2', 'site2': 'Y', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Z', 'site4': 'Y', 'site3': 'Z', 'site1': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Z', 'site3': 'I2', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site5': 'X', 'site7': 'I2', 'site4': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site2': 'Z', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'X', 'site4': 'Z', 'site6': 'Y', 'site8': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site7': 'I2', 'site2': 'Z', 'site6': 'Z', 'site4': 'Z', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site1': 'Z', 'site5': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'Y', 'site2': 'Z', 'site7': 'Y', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site2': 'I2', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'I2', 'site6': 'I2', 'site1': 'Z', 'site7': 'Y', 'site2': 'X', 'site5': 'X', 'site3': 'I2', 'site4': 'I2'}, {'site6': 'Y', 'site7': 'X', 'site4': 'I2', 'site8': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2'}, {'site5': 'X', 'site6': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Z', 'site8': 'I2', 'site2': 'Y', 'site4': 'I2', 'site6': 'X', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site1': 'I2', 'site8': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'Y', 'site2': 'I2', 'site3': 'Y', 'site6': 'Y', 'site1': 'Y', 'site8': 'I2', 'site4': 'I2', 'site7': 'I2'}, {'site7': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site1': 'Z', 'site5': 'X', 'site3': 'Y', 'site7': 'I2', 'site6': 'Z', 'site8': 'X', 'site2': 'I2', 'site4': 'I2'}, {'site2': 'I2', 'site8': 'I2', 'site7': 'I2', 'site3': 'X', 'site1': 'Z', 'site6': 'Z', 'site4': 'I2', 'site5': 'I2'}, {'site4': 'X', 'site1': 'X', 'site7': 'Z', 'site3': 'Z', 'site6': 'I2', 'site2': 'I2', 'site5': 'I2', 'site8': 'I2'}]
[-8.15, -9.51, 7.04, 7.93, -4.53, -1.51, -2.72, 9.15, -4.24, -7.48, 7.75, 7.54, -2.27, 6.16, 5.26, -1.74, -3.39, 4.35, -1.51, 6.9, 8.9, 8.45, -9.0, -6.14, -7.86] 
 [-8.15, -9.51, 7.04, 7.93, -4.53, -1.51, -2.72, 9.15, -4.24, -7.48, 7.75, 7.54, -2.27, 6.16, 5.26, -1.74, -3.39, 4.35, -1.51, 6.9, 8.9, 8.45, -9.0, -6.14, -7.86]
num:  500
Error happened!! Total difference:  12
[{'site2': 'Y', 'site5': 'X', 'site4': 'Y', 'site3': 'Y', 'site7': 'X', 'site1': 'X', 'site6': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'X', 'site5': 'I2', 'site6': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'I2', 'site7': 'Z', 'site8': 'I2', 'site1': 'I2', 'site3': 'Z', 'site2': 'X', 'site4': 'I2', 'site5': 'I2'}, {'site5': 'X', 'site3': 'I2', 'site2': 'I2', 'site7': 'Y', 'site4': 'I2', 'site1': 'X', 'site6': 'I2', 'site8': 'I2'}, {'site6': 'Y', 'site1': 'X', 'site4': 'Y', 'site7': 'Y', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site7': 'X', 'site4': 'Y', 'site3': 'Z', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Y', 'site4': 'X', 'site3': 'X', 'site5': 'I2', 'site7': 'Z', 'site8': 'Y', 'site6': 'I2', 'site2': 'I2'}, {'site1': 'Y', 'site5': 'X', 'site7': 'Z', 'site8': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2'}, {'site3': 'X', 'site8': 'X', 'site5': 'X', 'site6': 'Z', 'site7': 'I2', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2'}, {'site2': 'I2', 'site7': 'Z', 'site5': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site1': 'Z', 'site4': 'Y', 'site2': 'X', 'site3': 'Z', 'site7': 'I2', 'site6': 'X', 'site5': 'Y', 'site8': 'I2'}, {'site2': 'I2', 'site3': 'X', 'site6': 'Z', 'site5': 'Y', 'site7': 'X', 'site1': 'Z', 'site8': 'X', 'site4': 'I2'}, {'site6': 'X', 'site7': 'I2', 'site3': 'I2', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site7': 'I2', 'site3': 'X', 'site6': 'Y', 'site1': 'Y', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site3': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'X', 'site7': 'I2', 'site1': 'I2', 'site6': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2'}, {'site3': 'Z', 'site4': 'Z', 'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site7': 'I2', 'site3': 'X', 'site6': 'Z', 'site4': 'Z', 'site1': 'Z', 'site2': 'I2', 'site5': 'I2'}, {'site3': 'X', 'site6': 'Y', 'site1': 'Y', 'site2': 'Z', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'I2', 'site3': 'Y', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site2': 'Z', 'site6': 'Z', 'site8': 'Y', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site7': 'Z', 'site8': 'Z', 'site4': 'Y', 'site6': 'X', 'site3': 'I2', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2'}, {'site5': 'Z', 'site4': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}]
[-7.78, -9.95, 1.76, -2.44, -9.74, 6.95, 7.46, -9.62, 2.82, 2.46, -8.53, -9.95, 3.16, -8.9, 9.96, -5.18, 4.4, 7.22, -2.08, -8.56, -7.98, 3.56, -3.97, -4.22, 5.48] 
 [-7.78, -9.95, 1.76, -2.44, -9.74, 6.95, 7.46, -9.62, 2.82, 2.46, -8.53, -9.95, 3.16, -8.9, 9.96, -5.18, 4.4, 7.22, -2.08, -8.56, -7.98, 3.56, -3.97, -4.22, 5.48]
 83%|██████████████████████████████████████████████████████████████████████▊              | 25/30 [22:57<07:10, 86.07s/it]num:  0
num:  0
num:  0
Error happened!! Total difference:  13
[{'site8': 'Y', 'site4': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site7': 'Y', 'site5': 'X', 'site6': 'I2', 'site8': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2'}, {'site8': 'Z', 'site4': 'Y', 'site5': 'I2', 'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'X', 'site3': 'X', 'site4': 'I2', 'site6': 'Z', 'site7': 'I2', 'site1': 'I2', 'site2': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site3': 'I2', 'site1': 'Y', 'site5': 'I2', 'site2': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Z', 'site3': 'Z', 'site8': 'I2', 'site7': 'Z', 'site2': 'Y', 'site6': 'Y', 'site4': 'I2', 'site5': 'I2'}, {'site5': 'I2', 'site3': 'X', 'site6': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'Y', 'site5': 'X', 'site8': 'X', 'site6': 'X', 'site1': 'Z', 'site2': 'Y', 'site4': 'I2', 'site7': 'I2'}, {'site2': 'Z', 'site7': 'I2', 'site5': 'Y', 'site8': 'I2', 'site1': 'X', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2'}, {'site7': 'I2', 'site6': 'Z', 'site4': 'Y', 'site3': 'X', 'site1': 'X', 'site8': 'X', 'site2': 'Y', 'site5': 'I2'}, {'site4': 'Z', 'site1': 'Y', 'site6': 'Y', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'Y', 'site4': 'I2', 'site8': 'Z', 'site1': 'Z', 'site5': 'I2', 'site7': 'Y', 'site2': 'Z', 'site6': 'I2'}, {'site5': 'Y', 'site3': 'X', 'site2': 'Y', 'site8': 'Y', 'site1': 'X', 'site6': 'Y', 'site7': 'I2', 'site4': 'I2'}, {'site6': 'I2', 'site1': 'Z', 'site2': 'I2', 'site3': 'X', 'site5': 'X', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'I2', 'site3': 'X', 'site8': 'Y', 'site5': 'Y', 'site7': 'Z', 'site1': 'Y', 'site2': 'I2', 'site6': 'I2'}, {'site2': 'Y', 'site3': 'X', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'I2', 'site5': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site8': 'I2', 'site1': 'X', 'site7': 'X', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2'}, {'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'X', 'site6': 'X', 'site5': 'I2', 'site4': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site7': 'I2'}, {'site1': 'Y', 'site3': 'X', 'site6': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Z', 'site8': 'Z', 'site7': 'I2', 'site6': 'Z', 'site2': 'Y', 'site3': 'I2', 'site5': 'I2', 'site4': 'I2'}, {'site1': 'X', 'site8': 'Z', 'site6': 'Z', 'site3': 'X', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site8': 'Z', 'site5': 'Z', 'site7': 'Z', 'site4': 'Y', 'site3': 'Y', 'site1': 'I2', 'site2': 'X', 'site6': 'I2'}, {'site6': 'X', 'site4': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}]
[-9.99, 6.58, -2.53, 4.36, 8.04, -5.33, -3.59, 7.09, 6.8, -9.01, -7.08, 5.08, -5.53, 7.75, -2.72, -9.75, 7.75, -8.68, 5.89, -1.76, -3.95, -8.56, -8.78, -6.73, -6.11, 9.53] 
 [-9.99, 6.58, -2.53, 4.36, 8.04, -5.33, -3.59, 7.09, 6.8, -9.01, -7.08, 5.08, -5.53, 7.75, -2.72, -9.75, 7.75, -8.68, 5.89, -1.76, -3.95, -8.56, -8.78, -6.73, -6.11, 9.53]
Error happened!! Total difference:  14
[{'site6': 'Y', 'site4': 'I2', 'site5': 'I2', 'site1': 'X', 'site2': 'I2', 'site3': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site5': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'X', 'site3': 'Z', 'site8': 'Y', 'site6': 'Z', 'site1': 'I2', 'site4': 'X', 'site5': 'Z', 'site7': 'I2'}, {'site4': 'Z', 'site1': 'X', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'Y', 'site1': 'Y', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Y', 'site4': 'I2', 'site7': 'Z', 'site5': 'X', 'site3': 'Y', 'site2': 'X', 'site6': 'I2', 'site8': 'I2'}, {'site6': 'Z', 'site3': 'Y', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'I2', 'site7': 'X', 'site4': 'X', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'X', 'site6': 'I2', 'site2': 'Y', 'site5': 'Y', 'site1': 'X', 'site4': 'X', 'site3': 'I2', 'site7': 'I2'}, {'site7': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site5': 'X', 'site6': 'I2', 'site2': 'Z', 'site7': 'Y', 'site8': 'Y', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2'}, {'site5': 'I2', 'site6': 'Y', 'site7': 'X', 'site1': 'Z', 'site4': 'X', 'site3': 'Z', 'site8': 'I2', 'site2': 'I2'}, {'site8': 'Z', 'site4': 'X', 'site2': 'I2', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site3': 'Z', 'site1': 'I2', 'site2': 'Y', 'site4': 'Z', 'site6': 'Z', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'Z', 'site7': 'I2', 'site1': 'X', 'site3': 'Y', 'site4': 'Z', 'site2': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site6': 'I2', 'site8': 'Y', 'site7': 'Z', 'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2'}, {'site6': 'X', 'site5': 'X', 'site2': 'Y', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site6': 'Y', 'site7': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site4': 'I2', 'site6': 'Z', 'site2': 'X', 'site1': 'Y', 'site5': 'X', 'site3': 'X', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Y', 'site7': 'Y', 'site3': 'Z', 'site5': 'Y', 'site2': 'I2', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site6': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site6': 'Y', 'site2': 'Z', 'site8': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site6': 'X', 'site1': 'Z', 'site8': 'I2', 'site7': 'Y', 'site4': 'X', 'site5': 'Y', 'site2': 'I2', 'site3': 'I2'}, {'site7': 'X', 'site5': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}]
[-3.58, -8.47, 7.24, -4.47, 5.13, -9.56, 6.36, 7.65, -6.89, 1.87, 6.84, 2.55, -6.93, 2.94, 4.69, 5.05, 4.37, 6.71, 7.65, 6.79, 8.0, 7.44, 8.2, -4.28, 9.98, 1.87] 
 [-3.58, -8.47, 7.24, -4.47, 5.13, -9.56, 6.36, 7.65, -6.89, 1.87, 6.84, 2.55, -6.93, 2.94, 4.69, 5.05, 4.37, 6.71, 7.65, 6.79, 8.0, 7.44, 8.2, -4.28, 9.98, 1.87]
num:  500
num:  500
num:  500
Error happened!! Total difference:  15
[{'site3': 'I2', 'site2': 'X', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'X', 'site8': 'X', 'site3': 'I2', 'site6': 'Y', 'site2': 'X', 'site5': 'Y', 'site7': 'Z', 'site4': 'I2'}, {'site1': 'Z', 'site7': 'I2', 'site8': 'Y', 'site6': 'X', 'site4': 'Y', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2'}, {'site3': 'Y', 'site2': 'Y', 'site5': 'X', 'site1': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site3': 'X', 'site7': 'Y', 'site1': 'Y', 'site2': 'I2', 'site5': 'X', 'site6': 'I2', 'site8': 'I2'}, {'site1': 'I2', 'site4': 'Z', 'site3': 'I2', 'site2': 'I2', 'site7': 'I2', 'site8': 'Y', 'site5': 'I2', 'site6': 'I2'}, {'site8': 'I2', 'site1': 'Y', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site1': 'Z', 'site2': 'I2', 'site4': 'Y', 'site7': 'Y', 'site8': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'X', 'site3': 'Y', 'site8': 'Z', 'site4': 'Y', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site7': 'Y', 'site5': 'I2', 'site8': 'Z', 'site4': 'Y', 'site6': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2'}, {'site6': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'Y', 'site7': 'I2', 'site4': 'Y', 'site2': 'Z', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site4': 'I2', 'site6': 'Z', 'site1': 'Z', 'site8': 'X', 'site7': 'X', 'site5': 'Z', 'site2': 'I2', 'site3': 'I2'}, {'site3': 'I2', 'site5': 'Y', 'site8': 'I2', 'site2': 'Y', 'site7': 'Z', 'site1': 'I2', 'site6': 'X', 'site4': 'I2'}, {'site4': 'I2', 'site2': 'Z', 'site5': 'Z', 'site3': 'I2', 'site7': 'X', 'site1': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'Y', 'site5': 'X', 'site8': 'Z', 'site1': 'X', 'site7': 'I2', 'site6': 'Z', 'site2': 'Z', 'site4': 'I2'}, {'site5': 'X', 'site3': 'X', 'site2': 'I2', 'site1': 'Y', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'X', 'site5': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'X', 'site5': 'Z', 'site8': 'Y', 'site1': 'Z', 'site2': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site4': 'X', 'site7': 'Y', 'site8': 'Y', 'site1': 'Z', 'site6': 'X', 'site5': 'X', 'site2': 'I2', 'site3': 'I2'}, {'site1': 'X', 'site6': 'I2', 'site3': 'Y', 'site2': 'Z', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'Y', 'site6': 'Y', 'site7': 'Z', 'site4': 'Y', 'site2': 'Y', 'site8': 'Y', 'site1': 'I2', 'site3': 'I2'}, {'site3': 'X', 'site4': 'X', 'site1': 'I2', 'site2': 'Z', 'site7': 'X', 'site8': 'Z', 'site5': 'I2', 'site6': 'I2'}, {'site2': 'Y', 'site4': 'Z', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'I2', 'site8': 'I2', 'site7': 'X', 'site5': 'X', 'site1': 'Z', 'site6': 'Z', 'site2': 'I2', 'site4': 'I2'}]
[-6.88, -6.88, -7.53, -1.66, -6.32, 4.22, -3.36, -1.9, 9.64, 2.6, -8.66, 1.8, -8.24, 2.41, -9.48, -6.83, -4.04, -9.7, 2.5, -5.25, -6.57, -9.88, 4.86, 6.33, 6.45, -9.06] 
 [-6.88, -6.88, -7.53, -1.66, -6.32, 4.22, -3.36, -1.9, 9.64, 2.6, -8.66, 1.8, -8.24, 2.41, -9.48, -6.83, -4.04, -9.7, 2.5, -5.25, -6.57, -9.88, 4.86, 6.33, 6.45, -9.06]
 87%|█████████████████████████████████████████████████████████████████████████▋           | 26/30 [24:36<06:00, 90.01s/it]num:  0
Error happened!! Total difference:  16
[{'site2': 'X', 'site7': 'Z', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site4': 'X', 'site8': 'X', 'site5': 'Z', 'site1': 'Z', 'site6': 'Z', 'site2': 'I2', 'site3': 'I2', 'site7': 'I2'}, {'site1': 'I2', 'site4': 'I2', 'site2': 'Z', 'site3': 'Z', 'site7': 'Y', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site6': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'X', 'site7': 'X', 'site3': 'Y', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site5': 'Z', 'site8': 'I2', 'site2': 'Y', 'site7': 'X', 'site4': 'Y', 'site6': 'Y', 'site1': 'I2', 'site3': 'I2'}, {'site1': 'X', 'site8': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site1': 'X', 'site2': 'Y', 'site6': 'Z', 'site7': 'Y', 'site8': 'I2', 'site5': 'Y', 'site3': 'I2', 'site4': 'I2'}, {'site7': 'I2', 'site3': 'I2', 'site4': 'I2', 'site1': 'X', 'site6': 'I2', 'site2': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site2': 'Z', 'site3': 'Z', 'site7': 'X', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site2': 'I2', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site4': 'X', 'site7': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site7': 'Z', 'site4': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site6': 'Z', 'site3': 'Y', 'site8': 'X', 'site4': 'Y', 'site5': 'Y', 'site7': 'Z', 'site1': 'I2', 'site2': 'I2'}, {'site5': 'Z', 'site7': 'X', 'site8': 'X', 'site2': 'X', 'site6': 'X', 'site3': 'I2', 'site1': 'I2', 'site4': 'I2'}, {'site7': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site1': 'I2', 'site2': 'Z', 'site3': 'Z', 'site5': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Z', 'site8': 'Y', 'site6': 'Y', 'site5': 'Z', 'site3': 'Y', 'site1': 'I2', 'site4': 'I2', 'site7': 'I2'}, {'site2': 'I2', 'site1': 'Y', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site5': 'Y', 'site3': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site8': 'X', 'site1': 'Z', 'site6': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2'}, {'site1': 'Z', 'site8': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'I2', 'site1': 'Y', 'site4': 'Y', 'site7': 'I2', 'site6': 'Z', 'site2': 'X', 'site8': 'Z', 'site3': 'I2'}, {'site3': 'Z', 'site6': 'Y', 'site4': 'Z', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}]
[5.2, 1.68, 4.94, 4.4, -2.72, -6.88, 5.44, 6.98, 2.61, -8.06, -5.4, -6.49, 1.9, 5.37, 1.6, 3.6, 3.16, 4.48, -7.12, 4.94, 9.48, 9.03, 7.78, 4.88, 4.27, 8.16, -4.34] 
 [5.2, 1.68, 4.94, 4.4, -2.72, -6.88, 5.44, 6.98, 2.61, -8.06, -5.4, -6.49, 1.9, 5.37, 1.6, 3.6, 3.16, 4.48, -7.12, 4.94, 9.48, 9.03, 7.78, 4.88, 4.27, 8.16, -4.34]
Error happened!! Total difference:  17
[{'site1': 'Z', 'site8': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site4': 'X', 'site1': 'Z', 'site3': 'I2', 'site6': 'Z', 'site5': 'Y', 'site8': 'I2', 'site7': 'X', 'site2': 'I2'}, {'site8': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site8': 'X', 'site2': 'I2', 'site7': 'Y', 'site6': 'I2', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2'}, {'site2': 'I2', 'site3': 'X', 'site4': 'X', 'site5': 'X', 'site6': 'X', 'site1': 'Y', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site1': 'X', 'site7': 'X', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site1': 'X', 'site8': 'I2', 'site4': 'X', 'site2': 'X', 'site6': 'Y', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site7': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site2': 'Y', 'site5': 'I2', 'site7': 'I2', 'site8': 'X', 'site4': 'Y', 'site6': 'Y', 'site1': 'I2'}, {'site2': 'Y', 'site6': 'Y', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'X', 'site5': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'I2', 'site2': 'X', 'site7': 'I2', 'site8': 'X', 'site4': 'X', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2'}, {'site7': 'I2', 'site3': 'X', 'site4': 'X', 'site2': 'Z', 'site1': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site6': 'X', 'site1': 'Y', 'site8': 'X', 'site3': 'I2', 'site2': 'Z', 'site7': 'Z', 'site4': 'I2'}, {'site1': 'Y', 'site5': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'X', 'site1': 'X', 'site2': 'Z', 'site3': 'I2', 'site7': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site4': 'X', 'site2': 'I2', 'site3': 'Y', 'site1': 'Z', 'site8': 'Z', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site3': 'Y', 'site2': 'Y', 'site6': 'Z', 'site5': 'X', 'site1': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'X', 'site5': 'I2', 'site1': 'Y', 'site4': 'Y', 'site2': 'I2', 'site3': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'Z', 'site7': 'X', 'site6': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site8': 'I2'}, {'site1': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site1': 'I2', 'site7': 'Z', 'site5': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'I2', 'site6': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site5': 'X', 'site2': 'Y', 'site3': 'Y', 'site1': 'I2', 'site7': 'Y', 'site8': 'Z', 'site6': 'I2'}, {'site5': 'I2', 'site6': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site7': 'I2', 'site6': 'Z', 'site5': 'I2', 'site8': 'Z', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2'}]
[-2.51, 8.76, 4.71, 7.81, 8.37, 5.64, 3.35, 9.24, 4.39, 9.6, 4.71, -2.09, -8.3, -6.04, 8.93, 3.16, -8.6, 2.18, 9.14, -4.17, 8.74, 5.23, -1.54, -7.2, 2.01, 2.67, -3.32] 
 [-2.51, 8.76, 4.71, 7.81, 8.37, 5.64, 3.35, 9.24, 4.39, 9.6, 4.71, -2.09, -8.3, -6.04, 8.93, 3.16, -8.6, 2.18, 9.14, -4.17, 8.74, 5.23, -1.54, -7.2, 2.01, 2.67, -3.32]
num:  500
num:  500
num:  500
num:  500
 90%|████████████████████████████████████████████████████████████████████████████▌        | 27/30 [26:18<04:40, 93.37s/it]num:  0
num:  500
num:  500
num:  500
num:  500
Error happened!! Total difference:  18
[{'site4': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site3': 'I2', 'site7': 'I2', 'site6': 'Z', 'site2': 'X', 'site8': 'Z', 'site1': 'I2', 'site5': 'I2'}, {'site7': 'Z', 'site2': 'I2', 'site6': 'Z', 'site1': 'I2', 'site4': 'X', 'site5': 'I2', 'site3': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'X', 'site2': 'X', 'site4': 'Z', 'site8': 'I2', 'site5': 'I2', 'site7': 'Y', 'site1': 'I2', 'site3': 'I2'}, {'site4': 'X', 'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'X', 'site5': 'I2', 'site6': 'I2', 'site8': 'Z', 'site7': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2'}, {'site7': 'I2', 'site8': 'X', 'site1': 'I2', 'site4': 'Z', 'site5': 'I2', 'site2': 'I2', 'site6': 'Y', 'site3': 'I2'}, {'site5': 'I2', 'site1': 'X', 'site3': 'X', 'site2': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Z', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'Y', 'site1': 'Y', 'site7': 'X', 'site2': 'X', 'site5': 'X', 'site4': 'X', 'site8': 'X', 'site3': 'I2'}, {'site4': 'I2', 'site7': 'I2', 'site3': 'Y', 'site8': 'I2', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site4': 'Z', 'site6': 'I2', 'site2': 'Y', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site8': 'Z', 'site6': 'Z', 'site7': 'Y', 'site2': 'Y', 'site5': 'Y', 'site1': 'Z', 'site3': 'I2'}, {'site2': 'Z', 'site7': 'Y', 'site4': 'I2', 'site6': 'Y', 'site8': 'I2', 'site3': 'Z', 'site5': 'X', 'site1': 'I2'}, {'site2': 'Z', 'site4': 'Z', 'site6': 'X', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Y', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'X', 'site2': 'I2', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Z', 'site1': 'Y', 'site7': 'Y', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site7': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'I2', 'site4': 'Y', 'site2': 'Y', 'site7': 'X', 'site1': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site1': 'X', 'site4': 'Y', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'X', 'site8': 'X', 'site6': 'I2', 'site3': 'X', 'site4': 'Z', 'site5': 'I2', 'site1': 'Y', 'site7': 'I2'}, {'site3': 'I2', 'site7': 'Z', 'site4': 'X', 'site1': 'Y', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'I2', 'site7': 'Y', 'site6': 'I2', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site4': 'I2', 'site6': 'Z', 'site1': 'Y', 'site5': 'Y', 'site3': 'X', 'site7': 'Z', 'site2': 'I2', 'site8': 'I2'}, {'site6': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}]
[-7.32, 9.53, 8.17, 5.88, 5.98, 9.33, -5.74, -7.32, -4.68, 7.99, 3.34, 2.28, 7.16, -9.38, -1.52, 5.92, -1.99, 8.45, 5.49, 3.55, -8.64, -2.76, -8.27, -4.28, 3.69, -8.65, -5.84, -6.64] 
 [-7.32, 9.53, 8.17, 5.88, 5.98, 9.33, -5.74, -7.32, -4.68, 7.99, 3.34, 2.28, 7.16, -9.38, -1.52, 5.92, -1.99, 8.45, 5.49, 3.55, -8.64, -2.76, -8.27, -4.28, 3.69, -8.65, -5.84, -6.64]
[{'site3': 'Z', 'site5': 'X', 'site1': 'Z', 'site2': 'Z', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'Y', 'site4': 'Y', 'site6': 'X', 'site3': 'Z', 'site7': 'X', 'site8': 'Y', 'site1': 'I2', 'site2': 'I2'}, {'site1': 'Z', 'site6': 'I2', 'site3': 'Z', 'site4': 'X', 'site2': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'I2', 'site1': 'X', 'site7': 'I2', 'site4': 'X', 'site3': 'Z', 'site8': 'Z', 'site2': 'Z', 'site5': 'I2'}, {'site3': 'X', 'site8': 'I2', 'site1': 'Z', 'site6': 'I2', 'site5': 'X', 'site2': 'I2', 'site4': 'I2', 'site7': 'I2'}, {'site5': 'X', 'site2': 'I2', 'site4': 'I2', 'site7': 'X', 'site6': 'I2', 'site1': 'I2', 'site3': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site3': 'I2', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Z', 'site5': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'I2', 'site5': 'I2', 'site8': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site1': 'X', 'site5': 'I2', 'site7': 'I2', 'site2': 'I2', 'site4': 'I2', 'site3': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site2': 'Z', 'site7': 'X', 'site8': 'X', 'site4': 'I2', 'site5': 'I2', 'site1': 'I2', 'site3': 'I2', 'site6': 'I2'}, {'site7': 'I2', 'site1': 'X', 'site4': 'X', 'site6': 'Y', 'site5': 'Z', 'site3': 'X', 'site2': 'I2', 'site8': 'I2'}, {'site2': 'X', 'site5': 'I2', 'site3': 'Y', 'site1': 'Z', 'site8': 'Z', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'X', 'site4': 'I2', 'site6': 'I2', 'site1': 'X', 'site2': 'I2', 'site3': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site5': 'Z', 'site6': 'Y', 'site8': 'Y', 'site1': 'X', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2'}, {'site3': 'Z', 'site5': 'Y', 'site4': 'Z', 'site2': 'Y', 'site1': 'X', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site8': 'I2', 'site5': 'Z', 'site3': 'Z', 'site4': 'Y', 'site2': 'I2', 'site1': 'I2', 'site6': 'I2'}, {'site2': 'I2', 'site8': 'I2', 'site3': 'Z', 'site4': 'Z', 'site1': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site3': 'X', 'site5': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site5': 'Y', 'site6': 'Y', 'site4': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'X', 'site5': 'Y', 'site3': 'Y', 'site1': 'I2', 'site2': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site3': 'X', 'site6': 'I2', 'site2': 'Z', 'site4': 'I2', 'site1': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site1': 'Z', 'site5': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site2': 'X', 'site5': 'Y', 'site7': 'Y', 'site8': 'Z', 'site1': 'Y', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2'}, {'site6': 'Y', 'site3': 'I2', 'site7': 'Y', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site7': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site2': 'I2', 'site6': 'Z', 'site3': 'I2', 'site1': 'I2', 'site5': 'Y', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}]
[7.27, 8.51, 3.27, 9.19, -3.85, -5.71, -3.54, 9.7, -1.87, 5.8, 5.49, -7.5, -6.36, -6.6, 9.53, 2.89, 1.99, 2.17, 8.98, -7.77, -1.84, -9.04, -2.67, -6.75, -7.18, 9.17, 2.03, -7.81]
Difference is:  [15  4  4 13  4  9  4]  ----  [15  4  4 13  4  8  4]
Bond Total difference:  2
coefficients:  [7.27, 8.51, 3.27, 9.19, -3.85, -5.71, -3.54, 9.7, -1.87, 5.8, 5.49, -7.5, -6.36, -6.6, 9.53, 2.89, 1.99, 2.17, 8.98, -7.77, -1.84, -9.04, -2.67, -6.75, -7.18, 9.17, 2.03, -7.81]
 93%|███████████████████████████████████████████████████████████████████████████████▎     | 28/30 [28:01<03:12, 96.37s/it]num:  0
num:  0
num:  500
num:  500
Error happened!! Total difference:  19
[{'site7': 'Z', 'site8': 'Y', 'site5': 'Y', 'site2': 'X', 'site4': 'Z', 'site1': 'Z', 'site3': 'Y', 'site6': 'I2'}, {'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'X', 'site5': 'I2', 'site4': 'I2', 'site1': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'Y', 'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'I2', 'site7': 'X', 'site6': 'I2', 'site3': 'I2', 'site1': 'Y', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site3': 'Y', 'site6': 'Y', 'site1': 'Y', 'site8': 'I2', 'site2': 'Z', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site8': 'I2', 'site3': 'Y', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'X', 'site8': 'X', 'site4': 'Z', 'site6': 'I2', 'site3': 'I2', 'site1': 'I2', 'site2': 'I2', 'site7': 'I2'}, {'site3': 'X', 'site1': 'X', 'site2': 'Y', 'site6': 'X', 'site8': 'I2', 'site7': 'Z', 'site4': 'Z', 'site5': 'I2'}, {'site4': 'Y', 'site2': 'Z', 'site8': 'X', 'site1': 'Y', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site3': 'Y', 'site5': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site6': 'I2', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Y', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site6': 'Z', 'site5': 'X', 'site4': 'Y', 'site7': 'I2', 'site3': 'I2', 'site1': 'I2', 'site8': 'I2'}, {'site1': 'X', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site1': 'I2', 'site4': 'Y', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site7': 'I2', 'site6': 'Y', 'site8': 'X', 'site5': 'X', 'site2': 'Z', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2'}, {'site7': 'Z', 'site5': 'X', 'site2': 'Y', 'site1': 'X', 'site4': 'I2', 'site3': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'I2', 'site6': 'X', 'site8': 'Z', 'site7': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2'}, {'site8': 'Z', 'site6': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site8': 'Z', 'site2': 'X', 'site4': 'Y', 'site5': 'Z', 'site6': 'X', 'site3': 'Y', 'site1': 'Y', 'site7': 'I2'}, {'site3': 'I2', 'site7': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site1': 'I2', 'site6': 'Y', 'site3': 'X', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'I2', 'site6': 'X', 'site5': 'Z', 'site8': 'X', 'site7': 'Z', 'site4': 'I2', 'site1': 'Z', 'site3': 'I2'}, {'site7': 'X', 'site6': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site6': 'I2', 'site8': 'Z', 'site2': 'Y', 'site7': 'X', 'site1': 'X', 'site4': 'I2', 'site3': 'I2'}, {'site2': 'Z', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Z', 'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'Y', 'site6': 'Z', 'site4': 'X', 'site1': 'Y', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2'}]
[-6.22, -2.88, 5.23, -3.13, 4.88, -7.64, -9.53, -2.51, -5.39, 5.65, 6.75, 7.77, 9.47, -3.17, -7.63, -4.12, -5.75, 2.06, -9.24, -3.87, -2.36, -9.43, 3.09, -3.13, -3.21, -9.37, -9.57, -9.37, -3.36] 
 [-6.22, -2.88, 5.23, -3.13, 4.88, -7.64, -9.53, -2.51, -5.39, 5.65, 6.75, 7.77, 9.47, -3.17, -7.63, -4.12, -5.75, 2.06, -9.24, -3.87, -2.36, -9.43, 3.09, -3.13, -3.21, -9.37, -9.57, -9.37, -3.36]
[{'site7': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site7': 'Z', 'site8': 'Y', 'site3': 'Y', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'X', 'site6': 'X', 'site4': 'I2', 'site1': 'Z', 'site2': 'Y', 'site3': 'Y', 'site5': 'I2', 'site8': 'I2'}, {'site8': 'Y', 'site1': 'X', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site2': 'Y', 'site1': 'Z', 'site8': 'X', 'site3': 'Z', 'site7': 'Z', 'site5': 'I2', 'site4': 'Y', 'site6': 'I2'}, {'site3': 'Y', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site7': 'Y', 'site5': 'I2', 'site2': 'Z', 'site4': 'I2', 'site1': 'Y', 'site8': 'Z', 'site6': 'I2'}, {'site6': 'Y', 'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'I2', 'site4': 'X', 'site3': 'I2', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'I2', 'site4': 'Z', 'site5': 'X', 'site3': 'I2', 'site1': 'I2', 'site2': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site1': 'Z', 'site6': 'I2', 'site5': 'Y', 'site7': 'Y', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site6': 'Z', 'site8': 'I2', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site7': 'I2', 'site1': 'I2', 'site6': 'I2', 'site3': 'Z', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site1': 'Z', 'site2': 'I2', 'site3': 'Z', 'site4': 'I2', 'site8': 'Z', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site2': 'X', 'site3': 'I2', 'site5': 'X', 'site1': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'Y', 'site1': 'X', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'X', 'site5': 'Y', 'site6': 'X', 'site8': 'Z', 'site4': 'Z', 'site3': 'Z', 'site7': 'I2', 'site1': 'I2'}, {'site3': 'Z', 'site8': 'Z', 'site1': 'I2', 'site6': 'X', 'site7': 'Y', 'site4': 'I2', 'site2': 'I2', 'site5': 'I2'}, {'site4': 'X', 'site6': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Z', 'site3': 'Y', 'site5': 'Z', 'site1': 'I2', 'site2': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Z', 'site7': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site5': 'X', 'site4': 'Z', 'site7': 'I2', 'site3': 'X', 'site2': 'X', 'site6': 'Z', 'site8': 'Y', 'site1': 'I2'}, {'site6': 'Z', 'site8': 'X', 'site3': 'I2', 'site2': 'I2', 'site7': 'Y', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2'}, {'site7': 'Y', 'site4': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site1': 'Z', 'site5': 'I2', 'site6': 'X', 'site3': 'X', 'site4': 'Z', 'site2': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site3': 'Y', 'site2': 'I2', 'site1': 'Z', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site7': 'Y', 'site6': 'Y', 'site4': 'Y', 'site5': 'I2', 'site2': 'Z', 'site1': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site8': 'Y', 'site7': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2'}]
[3.88, 3.65, 2.37, 1.58, 4.69, 3.95, 7.88, -4.86, 3.79, 4.47, 1.9, 6.42, 2.91, -8.38, 7.83, 7.03, -2.03, 5.67, -5.44, 4.79, 3.66, 3.38, -6.64, 3.38, -4.68, 7.83, 5.32, 3.23, 6.64]
Difference is:  [17  4  4 20  4 10  4]  ----  [16  4  4 20  4 10  4]
Bond Total difference:  3
coefficients:  [3.88, 3.65, 2.37, 1.58, 4.69, 3.95, 7.88, -4.86, 3.79, 4.47, 1.9, 6.42, 2.91, -8.38, 7.83, 7.03, -2.03, 5.67, -5.44, 4.79, 3.66, 3.38, -6.64, 3.38, -4.68, 7.83, 5.32, 3.23, 6.64]
 97%|█████████████████████████████████████████████████████████████████████████████████▏  | 29/30 [29:51<01:40, 100.36s/it]num:  0
num:  0
num:  0
num:  0
num:  0
num:  0
Error happened!! Total difference:  20
[{'site2': 'X', 'site3': 'X', 'site8': 'Z', 'site4': 'X', 'site6': 'I2', 'site1': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site6': 'X', 'site3': 'X', 'site2': 'Z', 'site5': 'I2', 'site8': 'I2', 'site4': 'I2', 'site1': 'I2', 'site7': 'I2'}, {'site6': 'I2', 'site4': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site5': 'Y', 'site2': 'I2', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site6': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'I2', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site7': 'I2', 'site1': 'I2', 'site6': 'X', 'site2': 'Y', 'site5': 'I2', 'site3': 'X', 'site4': 'I2'}, {'site6': 'I2', 'site2': 'Z', 'site1': 'Z', 'site7': 'Z', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site7': 'Y', 'site3': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site7': 'X', 'site4': 'Y', 'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site5': 'X', 'site7': 'I2', 'site2': 'Z', 'site6': 'Y', 'site3': 'I2', 'site4': 'Y', 'site8': 'Z', 'site1': 'I2'}, {'site7': 'Y', 'site8': 'Z', 'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site1': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'Z', 'site5': 'X', 'site8': 'I2', 'site1': 'X', 'site2': 'X', 'site7': 'Y', 'site4': 'I2', 'site6': 'I2'}, {'site6': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site4': 'Z', 'site6': 'Y', 'site8': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site6': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site2': 'Z', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site3': 'X', 'site1': 'Y', 'site5': 'Y', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Z', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site7': 'I2', 'site8': 'Z', 'site4': 'Z', 'site6': 'Y', 'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2'}, {'site1': 'X', 'site5': 'Y', 'site7': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site1': 'Y', 'site2': 'Z', 'site5': 'X', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site2': 'Y', 'site6': 'Y', 'site3': 'Y', 'site8': 'I2', 'site1': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site1': 'Y', 'site4': 'X', 'site6': 'X', 'site2': 'X', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'X', 'site2': 'Y', 'site5': 'Z', 'site3': 'Z', 'site1': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'X', 'site1': 'X', 'site6': 'I2', 'site2': 'Y', 'site7': 'Y', 'site8': 'Y', 'site3': 'I2', 'site5': 'I2'}, {'site6': 'X', 'site7': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}]
[-5.32, -9.06, -1.78, -3.03, -3.87, -4.56, -5.52, 3.68, -3.68, 5.28, 2.77, 4.99, -6.56, -2.13, -8.15, -7.79, -4.09, 6.32, -9.06, 2.87, -4.0, -8.31, -5.41, 6.33, -7.64, -6.29, 8.22, 5.94, -6.51, -6.18] 
 [-5.32, -9.06, -1.78, -3.03, -3.87, -4.56, -5.52, 3.68, -3.68, 5.28, 2.77, 4.99, -6.56, -2.13, -8.15, -7.79, -4.09, 6.32, -9.06, 2.87, -4.0, -8.31, -5.41, 6.33, -7.64, -6.29, 8.22, 5.94, -6.51, -6.18]
[{'site4': 'Z', 'site2': 'Y', 'site8': 'Y', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site3': 'X', 'site4': 'Y', 'site7': 'I2', 'site6': 'Y', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site7': 'Z', 'site3': 'X', 'site2': 'Y', 'site1': 'Y', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site2': 'I2', 'site1': 'Z', 'site3': 'Z', 'site4': 'Y', 'site7': 'I2', 'site5': 'Z', 'site8': 'X', 'site6': 'I2'}, {'site3': 'I2', 'site5': 'Z', 'site4': 'X', 'site1': 'I2', 'site6': 'Z', 'site2': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site2': 'I2', 'site6': 'X', 'site1': 'Z', 'site7': 'I2', 'site3': 'Y', 'site4': 'I2', 'site8': 'I2'}, {'site5': 'Y', 'site2': 'X', 'site1': 'Y', 'site4': 'I2', 'site8': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site1': 'I2', 'site2': 'Y', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'I2', 'site5': 'X', 'site6': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site8': 'I2'}, {'site8': 'I2', 'site4': 'Y', 'site2': 'Z', 'site6': 'Z', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site5': 'Y', 'site8': 'X', 'site3': 'I2', 'site7': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site6': 'I2'}, {'site5': 'Y', 'site2': 'I2', 'site1': 'Z', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Z', 'site8': 'Z', 'site1': 'X', 'site7': 'Y', 'site2': 'Z', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site6': 'X', 'site7': 'I2', 'site5': 'Y', 'site2': 'Y', 'site4': 'Y', 'site1': 'I2', 'site3': 'I2', 'site8': 'I2'}, {'site4': 'I2', 'site1': 'X', 'site8': 'X', 'site6': 'Y', 'site7': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2'}, {'site4': 'Z', 'site8': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site7': 'X', 'site5': 'I2', 'site6': 'X', 'site2': 'Y', 'site8': 'I2', 'site3': 'Y', 'site1': 'I2', 'site4': 'I2'}, {'site6': 'X', 'site2': 'I2', 'site7': 'Z', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site8': 'I2'}, {'site8': 'X', 'site3': 'Y', 'site2': 'X', 'site5': 'I2', 'site4': 'I2', 'site1': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'X', 'site4': 'Z', 'site7': 'I2', 'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site4': 'I2', 'site7': 'Y', 'site3': 'Y', 'site1': 'Y', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site6': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'I2', 'site1': 'X', 'site3': 'Z', 'site7': 'X', 'site6': 'Z', 'site4': 'Y', 'site5': 'I2', 'site8': 'I2'}, {'site6': 'Y', 'site4': 'Z', 'site2': 'Y', 'site3': 'I2', 'site1': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'Y', 'site8': 'Y', 'site2': 'Y', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site7': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site4': 'I2', 'site8': 'Y', 'site2': 'Z', 'site1': 'I2', 'site7': 'Y', 'site6': 'Z', 'site5': 'I2', 'site3': 'I2'}, {'site4': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site8': 'X', 'site7': 'Y', 'site1': 'Y', 'site5': 'X', 'site6': 'Z', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2'}]
[-1.69, -2.44, 1.84, -4.17, 6.63, -7.17, -8.39, -7.95, 4.69, 8.0, 7.65, -2.4, 8.31, -2.81, -9.49, 5.09, -6.93, -8.67, -8.56, 8.43, 9.42, -3.16, 5.71, -1.91, -8.55, 8.8, 8.87, -2.67, -2.59, -6.26]
Difference is:  [17  4  4 20  4 10  4]  ----  [16  4  4 20  4 10  4]
Bond Total difference:  4
coefficients:  [-1.69, -2.44, 1.84, -4.17, 6.63, -7.17, -8.39, -7.95, 4.69, 8.0, 7.65, -2.4, 8.31, -2.81, -9.49, 5.09, -6.93, -8.67, -8.56, 8.43, 9.42, -3.16, 5.71, -1.91, -8.55, 8.8, 8.87, -2.67, -2.59, -6.26]
num:  500
num:  500
num:  500
100%|█████████████████████████████████████████████████████████████████████████████████████| 30/30 [31:45<00:00, 63.51s/it]
Total Number of Hamiltonian errors:  20
Total Number of Bond errors:  4
"""

In [47]:
import re
import ast

def extract_hamiltonians_and_coefficients(log_text):
    hamiltonians = []
    coefficients = []
    
    # Regular expression to capture Hamiltonians and coefficients
    hamiltonian_pattern = re.compile(r"\[{'site.*?\}\]", re.DOTALL)
    coefficient_pattern = re.compile(r"coefficients:\s*\[([-\d.,\s]+)\]")

    # Extracting Hamiltonians
    hamiltonian_matches = hamiltonian_pattern.findall(log_text)
    for match in hamiltonian_matches:
        try:
            # Convert string representation of list of dictionaries to actual list of dicts
            hamiltonians.append(ast.literal_eval(match))
        except Exception as e:
            print(f"Error parsing Hamiltonian: {e}")

    # Extracting Coefficients
    coefficient_matches = coefficient_pattern.findall(log_text)
    for match in coefficient_matches:
        try:
            # Convert string representation of list of floats to actual list of floats
            coefficients.append(ast.literal_eval(match))
        except Exception as e:
            print(f"Error parsing Coefficient: {e}")

    return hamiltonians, coefficients


In [ ]:
hamiltonians, coefficients = extract_hamiltonians_and_coefficients(input_string)
    
# Display the extracted data
print(len(coefficients), len(hamiltonians))
for c in coefficients:
    print(c)

for i , ham in enumerate(hamiltonians):
    
    coeffs = [random.randint(1, 20) for _ in range(len(ham))]
    hamiltonian_temp = construct_hamiltonian(ham,coeffs)
    ttns_root = construct_reference_tree()
    ham_t = hamiltonian_temp.pad_with_identities(ttns_root)

    ttno_ham = ptn.TTNO.from_hamiltonian(ham_t, ttns_root)
    total_tensor = ham_t.to_tensor(ttns_root).operator
    ttno_svd = ptn.TTNO.from_tensor(ttns_root,
                                    total_tensor,
                                    leg_dict,
                                    mode=ptn.Decomposition.tSVD)
    ham_dim = obtain_bond_dimensions(ttno_ham)
    svd_dim = obtain_bond_dimensions(ttno_svd)


    

    if np.any(ham_dim > svd_dim):
        print(i, ham_dim, svd_dim, "still broken for", ham)


In [ ]:
from numpy.random import default_rng
import random


rng = default_rng()

n = 10
random_number = random.choice([*range(-n, 0), *range(1, n + 1)])
random_number